# Applied Data Science Capstone

## Neighborhood similarity in major cities

#### In completion of requirements for the IBM Data Science Professional Certificate on Coursera

Daniel Nezich

<hr>

Cities compared:
* Canada
    * Toronto
    * Montréal
    * Vancouver
    * Halifax
* United States of America
    * New York City
    * Boston
    * Chicago
    * San Fransisco
* France
    * Paris
* England
    * London

First, get a list of Forward Sortation Areas as proxies for neighborhoods for cities in Canada.

In [2]:
# Data directory
DIR_DATA = 'Data/'
DIR_RESULTS = 'Results/'

# General
import pandas as pd
import numpy as np

# Postal Codes
import requests
# from bs4 import BeautifulSoup # had to install to environment in Anaconda
import lxml # had to install to environment in Anaconda, backdated to 4.6.1 (4.6.2 current) for pandas read_html()
import html5lib # had to install to environment in Anaconda (1.1 current) for pandas read_html()

# Geocoding
import config
from geopy.geocoders import Here
from geopy import distance
import matplotlib.pyplot as plt
%matplotlib inline

# Census Features
import folium
import matplotlib.pyplot as plt
%matplotlib inline

# Venue Features
import config
import importlib
import requests
import matplotlib.pyplot as plt
%matplotlib inline

# K-Means Clustering
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
%matplotlib inline

# Optimal K-Means Model
import folium
import matplotlib.cm as cm
import matplotlib.colors
from folium.features import DivIcon



# Line profiling  # https://mortada.net/easily-profile-python-code-in-jupyter.html
import line_profiler
%load_ext line_profiler

# Parsing
import geopandas
# import descartes

# Map display
import folium
import matplotlib.pyplot as plt
%matplotlib inline
import branca.element as bre
from branca.colormap import LinearColormap

# FSA correspondence
import warnings
warnings.filterwarnings("ignore")
import time
#import shapely
import dill # for saving/loading results to save on computation time

## Data structure

To facilitate automation of iteration through cities, we will organize each city's information within a dict and arrange those dicts in a list:

'cities' is a list of dicts that each have entries:

    name:        The name of the city
    group:       The name of the city group; all groups have the same geographic data source and processing procedure
    gdf:         A geopandas geodataframe, collects geometric unit information like area, population, etc.
    geojson:     The json representation of the geometry in gdf, with only required properties
    centroid:    [lat,long] of the centriod of all geometries in gdf
    bounds:      List of two [lat,long] lists describing southwest and northeast corners of a gdf bounding box
    df_ven:      A pandas dataframe, collects venue information, keyed to DAUID
    df_features: A pandas dataframe with only final clustering features, keyed to DAUID

In [51]:
cities = []

In [52]:
citynames_CA = ['Toronto','Montréal','Vancouver','Halifax']
citygroup_CA = 'Canada'
for cityname in citynames_CA:
    if not cityname in [city['name'] for city in cities]:
        cities.append({'name':cityname, 'group':citygroup_CA})

### Geographic Processing (Canada)

[Statistics Canada](https://www.statcan.gc.ca/eng/start) provides [boundary files](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm) for several scales of geographic division for which census statistics are reported.  Each bounded area is labeled with descriptive [attributes](https://www150.statcan.gc.ca/n1/pub/92-160-g/2016002/tbl/tbl_4.13-eng.htm).  The smallest area for which all statistics are reported is the Dissemination Area, the files for which include the following heirarchical labels:

Abbrev. | Name | Population | Description
---|---|---|---
DA | Dissemination Area | 400-700 | Composed of Dissemination Blocks.  Smallest standard geographic area for which all census data are disseminated
ADA | Aggregate Dissemination Area | 5,000-15,000 | Cover the entire country.  Respect PR, CD, CMA/CA, CT boundaries.
CT | Census Tract | <10,000 | Located in CMA/CA with core population >=50,000
CMA | Census Metropolitan Area / Census Agglomeration | >50,000 / >10,000 | Formed by adjacent municipalities around core with high degree of integration
CSD | Census Subdivision | | Municipality or equivalent
CCS | Consolidated Census Subdivision | | Adjacent CSD within same CD
CD | Census Division | | Neighboring municipalities linked for regional planning and service management
ER | Economic Region | | Group of complete CD (one exception in Ontario) created for regional economic analysis
PR | Province/Territory | | Province or Territory

We will begin by looking at Distribution Areas as proxies for neighborhoods for cities in Canada.  In previous work where the Forward Sortation Areas (first three characters of the postal code) were used as neighborhood proxies, the sizes of many areas were quite large (several kilometers across) and therefore are likely internally non-homogeneous from a features perspective at the walking-distance (500 m) length scale.  To convert our chosen geographic areas to neighborhood names we could use the [FSA boundary file](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm) intersection with the DA to find the appropriate FSA, then look up the associated neighborhood names from [this](https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods) Wikipedia page.  A source for neighborhood boundary files remains an open question.

File lda_000b16g_e.gml was downloaded from the [Statistics Canada: Boundary Files](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm) website.

Exploring the gml file and computing the area and centroid of the distribution areas can be done using the [geopandas module](https://geopandas.org/).  Geopandas builds upon [osgeo](https://gdal.org/python/index.html) which can also be used to explore and compute with the gml file, but in testing geopandas was 46 times faster due to vectorization of many calculations compared to a naive approach (see Appendix).

Latitude and Longitude need to be obtained from a projection with those units, e.g. [EPSG-4326](https://epsg.io/4326).  Area can be calculated from an equal-area projection, e.g. [EPSG-6931](https://epsg.io/6931) (though a geodesic area calculation would be more accurate for larger regions, that would have to come from an additional package such as [proj](https://proj.org/), but since all regions here are small such the curvature of the earth is negligible, and altitude indtroduces an additional error likely comparable or larger to the curvature error, we will proceed with a simpler equal-area projection calculation).  The geometry is saved as text in the [Well-Known Text (WKT)](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) representation.

#### Import using geopandas

In [ ]:
def GMLtoGDF(filename):
    gdf = geopandas.read_file(filename)
    gdf.rename_geometry('Geometry', inplace=True) # Default geometry column name is 'geometry'; changed for consistent capitalization of columns
    gdf.set_geometry('Geometry') # Renaming is insufficient; this sets special variable gdf.geometry = gdf['Geometry']
    gdf = gdf.set_crs(epsg=3347) # Needed only for FSA file, the others are 3347 and parsed correctly by geopandas, and the pdf in the zip file has the same projection parameters (FSA vs. DA, ADA, CT)
    gdf['Area'] = gdf['Geometry'].to_crs(epsg=6931).area # Equal-area projection
    gdf['Centroid'] = gdf['Geometry'].centroid
    gdf['Geometry'] = gdf['Geometry'].to_crs(epsg=4326) # Latitude/Longitude representation
    gdf['Centroid'] = gdf['Centroid'].to_crs(epsg=4326) # Only the set geometry is converted with gdf.to_crs(); all other geometry-containing columns must be converted explicitly; here we convert all columns explicitly
    gdf = gdf.set_crs(epsg=4326) # The series and geodataframe can have separate crs; this was found necessary for the geopandas.union function to operate easily
    gdf['Centroid Latitude'] = gdf['Centroid'].geometry.y
    gdf['Centroid Longitude'] = gdf['Centroid'].geometry.x
    gdf.drop(columns = 'Centroid', inplace=True) # Because WKT Point cannot be serialized to JSON, we drop the Centroid column and keep only its float components
    return gdf

In [ ]:
%%time
gdf_CA_DA = GMLtoGDF(DIR_DATA+'lda_000b16g_e.gml') # CA for Canada, DA for Dissemination Area level
gdf_CA_DA.head(2)

#### Extract geojson for each city

We will use folium, which can display patches provided in a geojson format, which is trivial to get using geopandas.  Note that points cannot be serialized (what!?) so the centroid must be dropped.  Furthermore, when coloring the patches, folium uses a dataframe keyed to the patches (for the Choropleth class this is optional; for the GeoJson class the values can be extracted from the json directly).  Therefore for convenience we split the geodataframe into patch and data components.

We define some utility functions for conditionally extracting the geojson and selecting cities.

In [ ]:
def fillCity(city, gdf, fieldname, method='equals', names=None):
    '''Populates the city dict using data from gdf, selection method, and names
    
    Expects city keys: name, group
    Populates city keys: gdf, geojson, data, centroid, bounds, selection
        geojson contains keys DAUID, Geometry, Area
        centroid and bounds use [latitude,longitude] format
    
    fieldname must be a column name in gdf used for selection
    method determines how the fieldname entry is used for row selection, with respect to the city name
        'equals': names is None (uses city name) or a list of strings at least one of which must match fieldname exactly
        'contains': names is None (uses city name) or a list of strings at least one of which must appear in fieldname
    
    Will overwrite existing entries
    '''
    if names==None:
        names = [city['name']]
    gdf_new = selectRegion(gdf, fieldname, method, names)
    updateCity(city, gdf_new)
    city['selection'] = f'{fieldname} {method} {names}'

def selectRegion(gdf, fieldname, method='equals', names=None):
    '''Selects rows from gdf using method to compare entries in fieldname to names
    
    fieldname must be a column name in gdf used for selection
    method determines how the fieldname entry is used for row selection, with respect to the city name
        'equals': names is None (uses city name) or a list of strings at least one of which must match fieldname exactly
        'contains': names is None (uses city name) or a list of strings at least one of which must appear in fieldname
    '''
    if names==None:
        print('error: names must be a string or list of strings')
        return None
    if not type(names)==list:
        if not type(names)==str:
            print('error: names must be a string or list of strings')
            return None
        names = [names]
    
    select = False
    for name in names:
        if method=='equals':
            select = select | (gdf[fieldname]==name)
        elif method=='contains':
            select = select | gdf[fieldname].str.contains(name)
        else:
            print("error: method must be 'equals' or 'contains'.")
            return
    return gdf.loc[select,:]
    
def updateCity(city, gdf):
    '''Populates the city dict using data from gdf
    
    Expects city keys: name, group
    Populates city keys: gdf, geojson, data, centroid, bounds
        geojson contains gdf keys except Centroid
        centroid and bounds use [latitude,longitude] format
    
    Will overwrite existing entries
    '''
    city['gdf'] = gdf
    city['geojson'] = gdf.copy(deep=True).to_json()
    #city['data'] = gdf.drop(columns=['Geometry']).copy(deep=True)
    tmp = gdf.geometry.unary_union
    city['centroid'] = [tmp.centroid.y,tmp.centroid.x]
    tmp = tmp.envelope.boundary.coords.xy
    city['bounds'] = [[min(tmp[1]),min(tmp[0])],[max(tmp[1]),max(tmp[0])]]

def getCityByName(cityname):
    '''Returns the first city dict that matches cityname exactly'''
    for city in cities:
        if city['name']==cityname:
            return city
    return None

def getCityGroup(groupname):
    '''Returns the subset of cities where group matches groupname exactly'''
    ret = []
    for city in cities:
        if city['group']==groupname:
            ret.append(city)
    return ret

Now, we can simply populate the city dicts by selecting only Dissemination Areas with Consolidated Census Subdivision Name equal to the city name:

In [ ]:
for city in cities:
    if city['group']=='Canada':
        fillCity(city,gdf_CA_DA,'CCSNAME',method='equals')

We do a quick inspection of the first entries of the resulting dataframes as a sanity check:

In [ ]:
pd.set_option('display.max_columns', None)
for city in cities:
    print(city['name'],':',f"{city['gdf'].shape[0]}",'rows')
    display(city['gdf'].head(1))
    print()

We can look at how many census tracts have a given number of dissemination areas:

In [ ]:
def histDAinCT(cities):
    if type(cities)==dict:
        cities = [cities]
    n = len(cities)
    plt.figure(figsize=(4*n,3))
    for i, city in enumerate(cities):
        plt.subplot(1,n,i+1)
        plt.hist(city['gdf'].groupby(['CTNAME']).count()['DAUID'],bins=np.arange(-0.4,20.4,1),width=0.8)
        plt.xlabel('# Dissemination Areas in a Census Tract')
        plt.ylabel('Number of Census Tracts')
        plt.title(f"{city['name']}, {city['gdf'].shape[0]} Dissemination Areas")
        plt.xticks(np.arange(0,21,2))

In [ ]:
histDAinCT(getCityGroup('Canada'))

We can also look at the area distributions:

In [ ]:
print('DA Count per city:')
count_DA = [(city['name'], city['gdf'].shape[0]) for city in cities]
print(count_DA)

#### Visualize Dissemination Areas for each city

This will be done using Folium.

Note that there are some issues with intuitive rendering of multiple inline maps using Folium/Branca:
* The colorbar is hard-coded to 500 px width (see ColorMap._repr_html_ in the [documentation](https://github.com/python-visualization/branca/blob/master/branca/colormap.py))
* The colorbar is added to the first choropleth in the figure, not the choropleth that creates it.
* LinearColormap.to_step requires input for variable 'data' in order to set logarithmic scale, and still the colormap is not displayed logarithmically

To get around thes issues, we manually create a colormap to use for all choropleths, add a new div/map above the target row and add the colormap to it (because a colormap must be renedered from within a map), and add a second div for the title.  Actually fixing these issues would require editing the modules, a potential project for later.

In [49]:
def getCityBounds(cities, featurename):
    '''Returns the global bounds of column featurename across all cities
    
    Parameters
    ----------
    cities: dict or list of dicts as defined above
    featurename: str column name in cities[i]['gdf'] for which bounds will be obtained
    
    Returns
    -------
    (min, max) value across all cities
    '''
    if type(cities)==dict:
        cities = [cities]
    bounds = [[],[]]
    for city in cities:
        bounds[0].append(city['gdf'][featurename].min())
        bounds[1].append(city['gdf'][featurename].max())
    bounds[0] = min(bounds[0])
    bounds[1] = max(bounds[1])
    return bounds

def extendBound(bound,direction='up',method='nearestLeadingDigit',scale=10):
    '''Extend bound to next 'round' number
    
    Parameters
    ----------
    bound: float or float castable number or a list thereof
    direction: {'up','down',nonzero number} or a list of these values indicating the direction to round in
    method: str describing the extension method
        'nearestLeadingDigit': Bound is nearest numbers with leading digit followed by zeros
        'nearestPower': Bound is nearest integer power of scale (scale must be > 1).  For negative numbers, the sign and direction are reversed, the extension performed, then the sign of the result is reversed back.
        'nearestMultiple': Bound is nearest multiple of scale (scale must be > 0)
        'round': Bound is rounded using the default method
    scale: numeric as described in method options or a list thereof
    
    Returns
    -------
    float: the extended bound
    
    Notes
    -----
    All inputs, if not single-valued, must be list-castable and of equal length
    If all inputs are single-valued, the output is a float, otherwise it is a list of floats
    '''
    import numpy as np
    
    # Check and adjust the length of inputs
    unlist = False
    try:
        bound = list(bound)
    except:
        try:
            bound = [bound]
            unlist = True
        except:
            print("Input 'bound' must be numeric or convertible to list type.")
            return None
    try:
        if type(direction)==str:
            direction = [direction]
        direction = list(direction)
    except:
        try:
            direction = [direction]
        except:
            print("Input 'direction' must be a string or nonzero number or convertible to list type.")
            return None
    try:
        if type(method)==str:
            method = [method]
        method = list(method)
    except:
        try:
            method = [method]
        except:
            print("Input 'method' must be a string or convertible to list type.")
            return None
    try:
        scale = list(scale)
    except:
        try:
            scale = [scale]
        except:
            print("Input 'scale' must be numeric or convertible to list type.")
            return None
    inputs = [bound, direction, method, scale]
    lengths = [len(i) for i in inputs]
    set_lengths = set(lengths)
    max_len = max(set_lengths)
    set_lengths.remove(1)
    if len(set_lengths)>1:
        print('Inputs must be of the same length or of length one.  See help(extendBound)')
        return None
    if max_len>1: # can this be converted to a looped statement?
        if len(bound)==1:
            bound = bound*max_len
        if len(direction)==1:
            direction = direction*max_len
        if len(method)==1:
            method = method*max_len
        if len(scale)==1:
            scale = scale*max_len
        unlist = False

    # If multiple methods are specified, recursively call this function for each method and reassemble results
    if len(bound)>1 and len(set(method))>1:
        ret = np.array([None for b in bound])
        for m in list(set(method)):
            ind = np.where(np.array(method)==m)
            ret[ind] = extendBound(list(np.array(bound)[ind]),list(np.array(direction)[ind]),m,list(np.array(scale)[ind]))
        return list(ret)
    
    # Convert direction to a logical array roundup
    try:
        roundup = [True if d=='up' else False if d=='down' else True if float(d)>0 else False if float(d)<0 else None for d in direction]
    except:
        print('direction must be "up", "down", or a non-negative number')
        return None
    if any([r==None for r in roundup]):
        print('direction must be "up", "down", or a non-negative number')
        return None
    
    # Cases for multiple methods handled above, return to string method
    method = method[0]
    
    # Execute the conversions
    if method=='nearestLeadingDigit':
        iszero = np.array(bound)==0
        isnegative = np.array(bound) < 0
        offsets = np.logical_xor(roundup, isnegative)
        power = [0 if z else np.floor(np.log10(abs(b))) for b, z in zip(bound, iszero)]
        firstdigit = [abs(b)//np.power(10,p) for b, p in zip(bound, power)]
        exceeds = [abs(b)>f*np.power(10,p) for b, f, p in zip(bound, firstdigit, power)]
        newbound = [abs(b) if not t else (f+o)*np.power(10,p) for b, t, n, f, o, p in zip(bound, exceeds, isnegative, firstdigit, offsets, power)]
        newbound = [-n if t else n for n, t in zip(newbound,isnegative)]
    elif method=='nearestPower':
        try:
            scale = [float(s) for s in scale]
            if any([s<=1 for s in scale]):
                print('scale should be greater than 1')
                return None
        except ValueError:
            print('scale should be a number or list of numbers greater than 1')
            return None
        isnegative = np.array(bound) < 0
        offsets = np.logical_xor(roundup, isnegative)
        roundfuns = [np.ceil if o else np.floor for o in offsets]
        newbound = [0 if b==0 else np.power(s, r(np.log10(abs(b))/np.log10(s))) for b, r, s in zip(bound,roundfuns,scale)]
        newbound = [-n if t else n for n, t in zip(newbound,isnegative)]
    elif method=='nearestMultiple':
        try:
            scale = [float(s) for s in scale]
            if any([s<=0 for s in scale]):
                print('scale should be greater than 0')
                return None
        except ValueError:
            print('scale should be a number or list of numbers greater than 0')
            return None
        roundfuns = [np.ceil if r else np.floor for r in roundup]
        newbound = [s*(r(b/s)) for b, r, s in zip(bound,roundfuns,scale)]
    elif method=='round':
        roundfuns = [np.ceil if r else np.floor for r in roundup]
        newbound = [f(b) for b, f in zip(bound, roundfuns)]
    else:
        print('Invalid method, see help(extendBound)')
        return None
    return newbound[0] if unlist else newbound

def extendBounds(bounds,method='nearestLeadingDigit',scale=10):
    if bounds[0]>bounds[1]:
        print('bounds must be ordered from least to greatest')
        return None    
    return extendBound(bounds,direction=['down','up'],method=method,scale=scale)

In [50]:
# Unit test of extendBounds
#   TODO: check out the builtin unittest module and convert this code to use that testing structure

# Get default arguments from https://stackoverflow.com/questions/12627118/get-a-function-arguments-default-value
import inspect
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

defaults = get_default_args(extendBound)
def test_extendBound(bounds,direction=defaults['direction'],method=defaults['method'],scale=defaults['scale'],expected=None): # default arguments taken from extendBound; not sure how to get defaults when not supplied
    output = extendBound(bounds,direction,method,scale)
    print('Input:',bounds,direction,method,scale,'  Output:',output,'  Expected:',expected,'  Passed:',output==expected)
    return output==expected

defaults = get_default_args(extendBounds)
def test_extendBounds(bounds,method=defaults['method'],scale=defaults['scale'],expected=None): # default arguments taken from extendBounds; not sure how to get defaults when not supplied
    output = extendBounds(bounds,method,scale)
    print('Input:',bounds,method,scale,'  Output:',output,'  Expected:',expected,'  Passed:',output==expected)
    return output==expected

print('Testing function extendBounds\n')

passed = True
print("Testing invalid method")
print("  Expect errors:")
passed = test_extendBounds([11,130],'invalid',expected=None) and passed
print()

print("Testing method 'nearestLeadingDigit'")
print("  Expect errors:")
passed = test_extendBounds([9,-930],'nearestLeadingDigit',expected=None) and passed
passed = test_extendBounds([-9,-930],'nearestLeadingDigit',expected=None) and passed
print("  Expect success:")
passed = test_extendBounds([11,130],'nearestLeadingDigit',expected=[10,200]) and passed
passed = test_extendBounds([11,130],'nearestLeadingDigit',-1,expected=[10,200]) and passed
passed = test_extendBounds([9,930],'nearestLeadingDigit',expected=[9,1000]) and passed
passed = test_extendBounds([-9,930],'nearestLeadingDigit',expected=[-9,1000]) and passed
passed = test_extendBounds([-990,-930],'nearestLeadingDigit',expected=[-1000,-900]) and passed
passed = test_extendBounds([-990,0.05],'nearestLeadingDigit',expected=[-1000,0.05]) and passed
passed = test_extendBounds([0,0.052],'nearestLeadingDigit',expected=[0,0.06]) and passed
print()

print("Testing method 'nearestPower'")
print("  Expect errors:")
passed = test_extendBounds([11,130],'nearestPower',-2,expected=None) and passed
passed = test_extendBounds([11,130],'nearestPower',0,expected=None) and passed
passed = test_extendBounds([11,130],'nearestPower',1,expected=None) and passed
passed = test_extendBounds([-11,-130],'nearestPower',10,expected=None) and passed
print("  Expect success:")
passed = test_extendBounds([11,130],'nearestPower',expected=[10,1000]) and passed
passed = test_extendBounds([10,100],'nearestPower',expected=[10,100]) and passed
passed = test_extendBounds([11,130],'nearestPower',1.1,expected=[10.834705943388395, 142.04293198443193]) and passed
passed = test_extendBounds([11,130],'nearestPower',2,expected=[8,256]) and passed
passed = test_extendBounds([11,130],'nearestPower',10,expected=[10,1000]) and passed
passed = test_extendBounds([11,130],'nearestPower',10.,expected=[10,1000]) and passed
passed = test_extendBounds([-11,130],'nearestPower',10,expected=[-100,1000]) and passed
passed = test_extendBounds([-5100,-130],'nearestPower',10,expected=[-10000,-100]) and passed
passed = test_extendBounds([-.0101,-0.00042],'nearestPower',10,expected=[-0.1,-0.0001]) and passed
passed = test_extendBounds([0,0.00042],'nearestPower',10,expected=[0,0.001]) and passed
print()

print("Testing method 'nearestMultiple'")
print("  Expect errors:")
passed = test_extendBounds([11,132],'nearestMultiple',-2,expected=None) and passed
passed = test_extendBounds([11,132],'nearestMultiple',0,expected=None) and passed
passed = test_extendBounds([0,-10],'nearestMultiple',100,expected=None) and passed
print("  Expect success:")
passed = test_extendBounds([11,132],'nearestMultiple',expected=[10,140]) and passed
passed = test_extendBounds([10,130],'nearestMultiple',expected=[10,130]) and passed
passed = test_extendBounds([11.55,132.55],'nearestMultiple',0.1,expected=[11.5,132.6]) and passed
passed = test_extendBounds([11.55,132.55],'nearestMultiple',1,expected=[11,133]) and passed
passed = test_extendBounds([11.55,132.55],'nearestMultiple',100,expected=[0,200]) and passed
passed = test_extendBounds([-11,132],'nearestMultiple',10,expected=[-20,140]) and passed
passed = test_extendBounds([-1121,-132],'nearestMultiple',10,expected=[-1130,-130]) and passed
passed = test_extendBounds([-10,-10],'nearestMultiple',10,expected=[-10,-10]) and passed
passed = test_extendBounds([-10,-10],'nearestMultiple',100,expected=[-100,0]) and passed
print()

print("Testing method 'round'")
print("  Expect errors:")
passed = test_extendBounds([-11.1,-132.1],'round',expected=None) and passed
print("  Expect success:")
passed = test_extendBounds([11.1,132.1],'round',expected=[11,133]) and passed
passed = test_extendBounds([10,130],'round',expected=[10,130]) and passed
passed = test_extendBounds([11.1,132.1],'round',-2,expected=[11,133]) and passed
passed = test_extendBounds([-11.1,132.1],'round',expected=[-12,133]) and passed
passed = test_extendBounds([-1100.1,-132.1],'round',expected=[-1101,-132]) and passed
print()

print("Testing array execution")
print("  Expect errors:")
print("  Expect success:")
print("    method")
passed = test_extendBound(1.5,'up','nearestLeadingDigit',4,expected=2) and passed
passed = test_extendBound(1.5,'up','nearestPower',4,expected=4) and passed
passed = test_extendBound(1.5,'up','nearestMultiple',4,expected=4) and passed
passed = test_extendBound(1.5,'up','round',4,expected=2) and passed
print("    direction")
passed = test_extendBound(1.5,'up','nearestMultiple',4,expected=4) and passed
passed = test_extendBound(1.5,0.1,'nearestMultiple',4,expected=4) and passed
passed = test_extendBound(1.5,'down','nearestMultiple',4,expected=0) and passed
passed = test_extendBound(1.5,-0.1,'nearestMultiple',4,expected=0) and passed
print("    broadcasting")
passed = test_extendBound([1.5],'up','nearestMultiple',1,expected=[2]) and passed
passed = test_extendBound([1.5,2.5],'up','nearestMultiple',1,expected=[2,3]) and passed
passed = test_extendBound(1.5,['up','down'],'nearestMultiple',1,expected=[2,1]) and passed
passed = test_extendBound(1.5,'up',['nearestLeadingDigit','nearestPower','nearestMultiple','round'],3,expected=[2,3,3,2]) and passed
passed = test_extendBound(1.5,'up','nearestMultiple',[1,5,10],expected=[2,5,10]) and passed
print()

print("All tests passed:",passed)

Testing function extendBounds

Testing invalid method
  Expect errors:
Invalid method, see help(extendBound)
Input: [11, 130] invalid 10   Output: None   Expected: None   Passed: True

Testing method 'nearestLeadingDigit'
  Expect errors:
bounds must be ordered from least to greatest
Input: [9, -930] nearestLeadingDigit 10   Output: None   Expected: None   Passed: True
bounds must be ordered from least to greatest
Input: [-9, -930] nearestLeadingDigit 10   Output: None   Expected: None   Passed: True
  Expect success:
Input: [11, 130] nearestLeadingDigit 10   Output: [10.0, 200.0]   Expected: [10, 200]   Passed: True
Input: [11, 130] nearestLeadingDigit -1   Output: [10.0, 200.0]   Expected: [10, 200]   Passed: True
Input: [9, 930] nearestLeadingDigit 10   Output: [9, 1000.0]   Expected: [9, 1000]   Passed: True
Input: [-9, 930] nearestLeadingDigit 10   Output: [-9, 1000.0]   Expected: [-9, 1000]   Passed: True
Input: [-990, -930] nearestLeadingDigit 10   Output: [-1000.0, -900.0]   Ex

In [ ]:
def mapCitiesAdjacent(cities,propertyname,title='',tooltiplabels=None):
    '''Displays cities in separate adjacent maps
    
    Cities dict as above
    Displayed as choropleth keyed to propertyname
    Header displays title and colormap labeled with keyed propertyname
    Tooltips pop up on mouseover showing properties listed in tooltiplabels
    
    Inspired by https://gitter.im/python-visualization/folium?at=5a36090a03838b2f2a04649d
    
    Assumes propertyname is identical in gdf and geojson
    '''
    if (not type(cities)==list) and type(cities)==dict:
        cities = [cities]
    
    f = bre.Figure()
    div_header = bre.Div(position='absolute',height='10%',width='100%',left='0%',top='0%').add_to(f)

    map_header = folium.Map(location=[0,0],control_scale=False,zoom_control=False,tiles=None,attr=False).add_to(div_header)
    div_header2 = bre.Div(position='absolute',height='10%',width='97%',left='3%',top='0%').add_to(div_header)
    html_header = '''<h3 align="left" style="font-size:16px;charset=utf-8"><b>{}</b></h3>'''.format(title)
    div_header2.get_root().html.add_child(folium.Element(html_header))

    vbounds = getCityBounds(cities,propertyname)
    vbounds[0] = 0
    vbounds = extendBounds(vbounds,'nearestLeadingDigit')
    
    cm_header = LinearColormap(
        colors=['yellow', 'orange', 'red'],
        index=None,
        vmin=vbounds[0],
        vmax=vbounds[1],
        caption=propertyname
        ).add_to(map_header) # .to_step(method='log', n=5, data=?), log has log labels but linear color scale

    for i, city in enumerate(cities):
        div_map = bre.Div(position='absolute',height='80%',width=f'{(100/len(cities))}%',left=f'{(i*100/len(cities))}%',top='10%').add_to(f)

        city_map = folium.Map(location=city['centroid'], control_scale=True)
        div_map.add_child(city_map)
        title_html = '''<h3 align="center" style="font-size:16px;charset=utf-8"><b>{}</b></h3>'''.format(city['name'])
        city_map.get_root().html.add_child(folium.Element(title_html))

        city_map.fit_bounds(city['bounds'])

        m = folium.GeoJson(
                    city['geojson'],
                    style_function=lambda feature: {
                        'fillColor': cm_header.rgb_hex_str(feature['properties'][propertyname]),
                        'fillOpacity': 0.8,
                        'color':'black',
                        'weight': 1,
                        'opacity': 0.2,
                    },
                    name=f'Choropleth_{i}'
                ).add_to(city_map)
        if not tooltiplabels==None:
            m.add_child(folium.features.GeoJsonTooltip(tooltiplabels))
    
    display(f)

In [ ]:
tooltip_DA = ['PRNAME','ERNAME','CDNAME','CCSNAME','CSDNAME','CMANAME','CDNAME','CTNAME','ADAUID','DAUID']
mapCitiesAdjacent(getCityGroup('Canada'),'Area','Canadian Cities, Dissemination Areas colored by square meters',tooltip_DA)

Toronto and Montréal look like reasonable areas.  Vancouver and Halifax could use some modification to make their areas of extent and density similar to Toronto and Montréal.

First let's examine Vancouver.  We extend the region to include adjacent Census Subdivisions that that remain near the city center.

In [ ]:
cityname = 'Vancouver'
gdf_select = gdf_CA_DA.loc[gdf_CA_DA['PRNAME'].str.contains('British Columbia'),:]
# print('ERNAME', gdf_select['ERNAME'].unique())
# print('CCSNAME', gdf_select['CCSNAME'].unique())
# display(getCityByName(cityname)['gdf'].iloc[0,:])
fillCity(getCityByName(cityname), gdf_select, 'CSDNAME', method='equals', names=['Vancouver','Burnaby','Richmond','New Westminster'])
mapCitiesAdjacent(getCityByName(cityname),'Area','Canadian Cities, Dissemination Areas colored by square meters',tooltip_DA)

Next let's examin Halifax, which requires removal of the large sparse areas.

In [ ]:
cityname = 'Halifax'
gdf_select = getCityByName(cityname)['gdf']
updateCity(getCityByName(cityname),gdf_select.loc[gdf_select['CTNAME']<123,:])
mapCitiesAdjacent(getCityByName(cityname),'CTNAME','Canadian Cities, Dissemination Areas colored by Census Tract ID',tooltip_DA)

We now review the selected areas:

In [ ]:
mapCitiesAdjacent(getCityGroup('Canada'),'Area','Canadian Cities, Dissemination Areas colored by square meters',tooltip_DA)

Because the Dissemination Areas are often small, we should also examine Census Tracts and Aggregate Dissemination Areas, which are the next largest reporting area designation.  Geometries for these areas can also be downloaded [here](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm).

We first save our existing work and then examine each grouping in detail.

In [ ]:
cities_DA = cities

#### Aggregate Dissemination Area

We collect the processing applied above for Dissemination Areas to see how ADAs compare.

In [ ]:
%%time
gdf_CA_ADA = GMLtoGDF(DIR_DATA+'lada000b16g_e.gml')

In [ ]:
gdf = gdf_CA_ADA
cities = []
citygroup = 'Canada'

for cityname in citynames_CA:
    if not cityname in [city['name'] for city in cities]:
        cities.append({'name':cityname, 'group':citygroup})

for city in getCityGroup(citygroup):
    city_da = []
    for c in cities_DA:
        if c['name']==city['name']:
            city_da = c
    fillCity(city,gdf,'ADAUID','equals',city_da['gdf']['ADAUID'].unique().tolist())

# Remove two large areas in Halifax
updateCity(getCityByName('Halifax'),getCityByName('Halifax')['gdf'].loc[~((getCityByName('Halifax')['gdf']['ADAUID']==12090050) | (getCityByName('Halifax')['gdf']['ADAUID']==12090010)),:])

cities_ADA = cities

tooltip_ADA = ['PRNAME','CDNAME','ADAUID']
mapCitiesAdjacent(getCityGroup(citygroup),'Area','Canadian Cities, Aggregate Dissemination Areas colored by square meters',tooltip_ADA)

In [ ]:
print('DA Count per city:')
print(count_DA)
print('ADA Count per city:')
count_ADA = [(city['name'], city['gdf'].shape[0]) for city in cities]
print(count_ADA)
print('DA/ADA Ratio per city:')
print([(a[0],float(f'{a[1]/b[1]:0.3}')) for a, b, in zip(count_DA,count_ADA)])

The ADAs have a reasonable size - the smallest are about 300 meters across, which is anly a bit smaller than our target walking distance.  The necessary processing was very similar.


#### Census Tract

In [ ]:
%%time
gdf_CA_CT = GMLtoGDF(DIR_DATA+'lct_000b16g_e.gml')

In [ ]:
gdf = gdf_CA_CT
cities = []
citygroup = 'Canada'

for cityname in citynames_CA:
    if not cityname in [city['name'] for city in cities]:
        cities.append({'name':cityname, 'group':citygroup})

for city in getCityGroup(citygroup):
    city_da = []
    for c in cities_DA:
        if c['name']==city['name']:
            city_da = c
    fillCity(city,gdf,'CTUID','equals',city_da['gdf']['CTUID'].unique().tolist())

# Remove two large areas in Halifax
#updateCity(getCityByName('Halifax'),getCityByName('Halifax')['gdf'].loc[~((tmp_gdf['ADAUID']==12090050) | (tmp_gdf['ADAUID']==12090010)),:])

cities_CT = cities

tooltip_ADA = ['PRNAME','CMANAME','CTUID']
mapCitiesAdjacent(getCityGroup(citygroup),'Area','Canadian Cities, Census Tracts colored by square meters',tooltip_ADA)

#### Forward Sortation Area

Forward Sortation Areas are not included in the Dissemination Area boundary file fields, though census data is provided sorted by FSA.  The FSA identifiers do not have detailed geographic correspondence, so we must seek an alternate way to determine what FSAs are of interest (within the bounds of a city).

Ideally we could use the Postal Code Correspondence File (PCCF) provided by Canada Post, but this is a restricted document (the PCCF goes into greater detail by linking the full postal code with Dissemination Areas/Blocks, and is therefore a privacy concern).

However, Statistics Canada provides boundary files for FSAs, so we can use these in combination with a DA boundary file to create a correspondence list.  The main caveat is that the FSA boundaries are not exactly the Canada Post FSA boundaries, but are derived from self-reported postal codes in census responses, massaged to respect DA and CT boundaries.  This is good for creation of a correspondence list and for consistency of census statistics reporting, but is not strictly accurate.

We begin by generating a correspondence list using the available boundary files.

For FSAs, there are no features to corresponde to the previous DAs, ADAs, and CTs.  We can resolve this in three ways: finding FSAs with a centroid within an arbitrary distance of any of the centroids of DAs for each city, creating an overlap dataframe using boundary files for FSAs and DAs and using that to select FSAs overlapping any DA in each city, or looking for an existing correspondence file.  The existing Postal Code Correspondence File (PCCF) is [no longer provided](https://www150.statcan.gc.ca/n1/en/catalogue/92-154-X) through Statistics Canada, and is behind logins or paywalls elsewhere online.

We proceed with the more exact solution of constructing our own correspondence file.  Luckily, geopandas has an intersection function that is just what we're looking for (I first found the overlay function, but that is over 100 times slower).

In [ ]:
%%time
gdf_CA_FSA = GMLtoGDF(DIR_DATA+'lfsa000b16g_e.gml')

In [ ]:
def intersectGDF(gdf1, keyfield1, gdf2, keyfield2, verbosity=1):
    '''Find the overlap matrix of two geodataframe geometries
    
    Parameters
    ----------
    gdf1: GeoDataFrame (must match crs of gdf2, will be utilized for vectorized overlap calculation)
    keyfield1: column name in gdf1 which uniquely identifies each row and will be used to label the results
    gdf2: GeoDataFrame (must match crs of gdf1, will be iterated over for overlap calculation)
    keyfield2: column name in gdf2 which uniquely identifies each row and will be used to label the results
    
    Returns
    -------
    gdf_union: Geodataframe containing columns of nonzero overlap geometries, corresponding gdf1[keyfield1], and corresponding gdf2[keyfield2], where only one value of gdf1[keyfield1] is selected which is the one with maximum overlap area
    times: List of execution times for each overlap calculation; len(times)=gdf2.shape[0]
    areas: List of pandas Series of overlap areas; len(areas)=gdf2.shape[0], len(areas[i])=gdf1.shape[0]
    
    Notes
    -----
    gdf1 and gdf2 must be set to the same crs
    Iterates over gdf2, which should have the larger number of rows of {gdf1,gdf2} in order to minimize required memory (assuming geometries are of roughly equal size)
    Uses GeoDataFrame.buffer(0) to correct geometries
    '''
    # Initialize the return variables
    gdf_union = geopandas.GeoDataFrame()
    times = []
    areas = []

    # Ensure we are computing with copies of the dataframes (though this should not be necessary as gdf1, gdf2 are never updated)
    gdf1 = gdf1.copy(deep=True)
    gdf2 = gdf2.copy(deep=True)
    
    # Create new dataframes to hold buffered geometries
    #  Buffered geometries can often prevent topology errors during overlap calculation, at the risk of discarding portions of the geometry
    gdf1b = gdf1.copy(deep=True)
    gdf2b = gdf2.copy(deep=True)

    start_time = time.time()
    gdf1b['Geometry'] = gdf1b['Geometry'].buffer(0)
    if verbosity>=1: print(f'Polygon conversion for {keyfield1} completed in {time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))}')

    start_time = time.time()
    gdf2b['Geometry'] = gdf2b['Geometry'].buffer(0)
    if verbosity>=1: print(f'Polygon conversion for {keyfield2} completed in {time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))}')

    exceptioncount = 0
    start_time = time.time()
    # TODO: create optimal modulus for verbose display
    verbosecount = 1 if verbosity>=2 else extendbounds(gdf2.shape[0],method='Log10')
    for i in range(gdf2.shape[0]):
        loop_start = time.time()
        try:
            gdf_tmp = gdf1['Geometry'].intersection(gdf2['Geometry'].iloc[i]) # Attempt using original geometries
        except (shapely.errors.TopologicalError): # This sometimes occurs
            if verbosity>=2: print(f'Handling exception at index {i}')
            exceptioncount += 1
            gdf_tmp = gdf1b['Geometry'].intersection(gdf2b['Geometry'].iloc[i]) # Use fallback buffered geometries

        areas.append(gdf_tmp.area)
        ind = np.argmax(areas[-1]) # The FSA boundaries respect DA boundaries according to the , so there is just one FSA associated with each DA, which should have significantly larger area than any other.
        gdf_tmp = geopandas.GeoSeries(gdf_tmp.iloc[ind],crs=gdf1['Geometry'].crs)
        gdf_tmp = geopandas.GeoDataFrame(geometry=gdf_tmp,crs=gdf_tmp.crs)
        gdf_tmp[keyfield1] = gdf1[keyfield1].iloc[ind]
        gdf_tmp[keyfield2] = gdf2[keyfield2].iloc[i]
        gdf_union = gdf_union.append(gdf_tmp,ignore_index=True)
        loop_end = time.time()
        times.append(loop_end-loop_start)
        if verbosity>=1 and (i%verbosecount==0 or i+1==gdf2.shape[0]):
            print(f'Processed row {i+1}/{gdf2.shape[0]}, {gdf_tmp.shape[0]} overlap found in {loop_end-loop_start:.1f} sec, {gdf_union.shape[0]} overlaps total in {time.strftime("%H:%M:%S", time.gmtime(loop_end-start_time))}, total exceptions {exceptioncount}')
    if verbosity>=1: print('Overlap processing complete')
    return gdf_union, times, areas

Now we compute the overlap between FSAs and DAs:

In [ ]:
try: # Results have been calculated previously, load them to save time
    with open(DIR_RESULTS+'GDF_FSA-DA.db','rb') as file:
        gdf_union = dill.load(file)
    with open(DIR_RESULTS+'GDF_FSA-DA_times.db','rb') as file:
        times = dill.load(file)
    with open(DIR_RESULTS+'GDF_FSA-DA_areas.db','rb') as file:
        areas = dill.load(file)
    print('Results loaded from file')

except IOError:  # Results not found in file, regenerate them
    gdf_union, times, areas = intersectGDF(gdf_CA_FSA,'CFSAUID',gdf_CA_DA,'DAUID',verbosity=1)

    with open(DIR_RESULTS+'GDF_FSA-DA.db','wb+') as file:
        dill.dump(gdf_union,file)
    with open(DIR_RESULTS+'GDF_FSA-DA_times.db','wb+') as file:
        dill.dump(times,file)
    with open(DIR_RESULTS+'GDF_FSA-DA_areas.db','wb+') as file:
        dill.dump(areas,file)
    print('Results saved to file')

The result

In [ ]:
gdf_union.shape

import dill

with open('GDF_FSA-DA.db','wb') as file:
    dill.dump(gdf_union,file)

with open('GDF_FSA-DA_times.db','wb') as file:
    dill.dump(times,file)
with open('GDF_FSA-DA_areas.db','wb') as file:
    dill.dump(areas,file)

In [ ]:
Do the 

In [ ]:
a = pd.DataFrame(areas)
#a.head(20)
print(a.shape)
print(gdf1.shape[0])
print(gdf2.shape[0])

We look at the results to make sure the selection criteria were reasonable (we check the ratio of the largest to the next largest area and ensure it is arbitrarily large).

In [ ]:
display(a.head(20))
asorted = np.array(areas)
asorted.sort(axis=1)
display(pd.DataFrame(asorted).head(20))
aratio = asorted[:,-2]/asorted[:,-1]
aratio[0:20]

In [ ]:
exponent = np.logspace(-10,0,11)
counts = [len(np.where(aratio>x)[0]) for x in exponent]
print('Ratio : Count')
print('-------------')
for x, c in zip(exponent, counts):
    print(x,':',c)

There is a five magnitude gap between tiny ratios where the errors are likely the result of minor coordinate differences, and large ratios indicating that two FSAs share significant overlap with a single DA, which is an apparent violation of the specified [rules]().

In [ ]:
rind = np.where(aratio>1e-3)[0]
rval = aratio[rind]
print('Index : Ratio')
print('-------------')
for i, v in zip(rind,rval):
    print(f'{i: >5}',':',f'{v:.3f}')

We can also look at whether the areas match 

Note that in the accompanying description file for the FSA boundaries (92-179-g2016001-eng.pdf), it notes that there are "twenty-one forward sortation areas which are not included in the boundary file because they were not the dominant FSA in a dissemination area."  This matching with the number of non-uniquely-associated FSA labels is expected to be a coincidence.

To complete the correspondence file, the boundaries involved in the anomalous ratios should be manually examined and corrected.

In [ ]:
for row in np.array(areas):
    row.sort()
    print(np.sort(row)[-2:])
    break
print(type(areas))

Now, this process might be easier if waterfronts are avoided, by using digital instead of cartographic boundary files, but it is a little late for that now (but might be useful to keep in mind for future projects).

Operation

    Polygon conversion for CFSAUID completed in 00:00:39
    Polygon conversion for DAUID completed in 00:00:28
    Processed row 1/56589, 1 overlap found in 0.3 sec, 1 overlaps total in 00:00:00, total exceptions 0
    Processed row 1001/56589, 1 overlap found in 0.2 sec, 1001 overlaps total in 00:05:02, total exceptions 0
    Processed row 2001/56589, 1 overlap found in 0.2 sec, 2001 overlaps total in 00:09:46, total exceptions 0
    Processed row 3001/56589, 1 overlap found in 0.3 sec, 3001 overlaps total in 00:14:15, total exceptions 0
    Processed row 4001/56589, 1 overlap found in 0.3 sec, 4001 overlaps total in 00:18:41, total exceptions 0
    Processed row 5001/56589, 1 overlap found in 0.2 sec, 5001 overlaps total in 00:23:00, total exceptions 0
    Processed row 6001/56589, 1 overlap found in 0.2 sec, 6001 overlaps total in 00:27:11, total exceptions 0
    Processed row 7001/56589, 1 overlap found in 0.3 sec, 7001 overlaps total in 00:31:21, total exceptions 0
    Processed row 8001/56589, 1 overlap found in 0.4 sec, 8001 overlaps total in 00:35:38, total exceptions 0
    Processed row 9001/56589, 1 overlap found in 0.2 sec, 9001 overlaps total in 00:39:53, total exceptions 0
    Processed row 10001/56589, 1 overlap found in 0.3 sec, 10001 overlaps total in 00:44:14, total exceptions 0
    Processed row 11001/56589, 1 overlap found in 0.3 sec, 11001 overlaps total in 00:48:30, total exceptions 0
    Processed row 12001/56589, 1 overlap found in 0.3 sec, 12001 overlaps total in 00:52:45, total exceptions 0
    Processed row 13001/56589, 1 overlap found in 0.3 sec, 13001 overlaps total in 00:57:00, total exceptions 0
    Processed row 14001/56589, 1 overlap found in 0.3 sec, 14001 overlaps total in 01:01:15, total exceptions 0
    Processed row 15001/56589, 1 overlap found in 0.2 sec, 15001 overlaps total in 01:05:32, total exceptions 0
    Processed row 16001/56589, 1 overlap found in 0.3 sec, 16001 overlaps total in 01:09:49, total exceptions 0
    Processed row 17001/56589, 1 overlap found in 0.3 sec, 17001 overlaps total in 01:14:16, total exceptions 0
    Processed row 18001/56589, 1 overlap found in 0.2 sec, 18001 overlaps total in 01:18:47, total exceptions 0
    Processed row 19001/56589, 1 overlap found in 0.2 sec, 19001 overlaps total in 01:23:40, total exceptions 0
    Processed row 20001/56589, 1 overlap found in 0.2 sec, 20001 overlaps total in 01:33:37, total exceptions 0
    Processed row 21001/56589, 1 overlap found in 0.3 sec, 21001 overlaps total in 01:37:55, total exceptions 0
    Processed row 22001/56589, 1 overlap found in 0.3 sec, 22001 overlaps total in 01:42:12, total exceptions 0
    Processed row 23001/56589, 1 overlap found in 0.2 sec, 23001 overlaps total in 01:46:26, total exceptions 0
    Processed row 24001/56589, 1 overlap found in 0.3 sec, 24001 overlaps total in 01:50:42, total exceptions 0
    Processed row 25001/56589, 1 overlap found in 0.3 sec, 25001 overlaps total in 01:54:55, total exceptions 0
    Processed row 26001/56589, 1 overlap found in 0.2 sec, 26001 overlaps total in 01:59:11, total exceptions 0
    Processed row 27001/56589, 1 overlap found in 0.2 sec, 27001 overlaps total in 02:03:25, total exceptions 0
    Processed row 28001/56589, 1 overlap found in 0.3 sec, 28001 overlaps total in 02:07:38, total exceptions 0
    Processed row 29001/56589, 1 overlap found in 0.2 sec, 29001 overlaps total in 02:11:54, total exceptions 0
    Processed row 30001/56589, 1 overlap found in 0.3 sec, 30001 overlaps total in 02:16:09, total exceptions 0
    Processed row 31001/56589, 1 overlap found in 0.3 sec, 31001 overlaps total in 02:20:25, total exceptions 0
    Processed row 32001/56589, 1 overlap found in 0.2 sec, 32001 overlaps total in 02:24:40, total exceptions 0
    Processed row 33001/56589, 1 overlap found in 0.3 sec, 33001 overlaps total in 02:29:04, total exceptions 0
    Processed row 34001/56589, 1 overlap found in 0.3 sec, 34001 overlaps total in 02:33:23, total exceptions 0
    Processed row 35001/56589, 1 overlap found in 0.2 sec, 35001 overlaps total in 02:37:43, total exceptions 0
    Processed row 36001/56589, 1 overlap found in 0.3 sec, 36001 overlaps total in 02:42:11, total exceptions 0
    Processed row 37001/56589, 1 overlap found in 0.3 sec, 37001 overlaps total in 02:46:40, total exceptions 0
    Processed row 38001/56589, 1 overlap found in 0.4 sec, 38001 overlaps total in 02:51:08, total exceptions 0
    Processed row 39001/56589, 1 overlap found in 0.3 sec, 39001 overlaps total in 02:56:51, total exceptions 0
    Processed row 40001/56589, 1 overlap found in 0.3 sec, 40001 overlaps total in 03:01:38, total exceptions 0
    Processed row 41001/56589, 1 overlap found in 0.3 sec, 41001 overlaps total in 03:06:38, total exceptions 0
    Processed row 42001/56589, 1 overlap found in 0.3 sec, 42001 overlaps total in 03:11:34, total exceptions 0
    Processed row 43001/56589, 1 overlap found in 0.2 sec, 43001 overlaps total in 03:16:47, total exceptions 0
    Processed row 44001/56589, 1 overlap found in 0.3 sec, 44001 overlaps total in 03:21:20, total exceptions 0
    Processed row 45001/56589, 1 overlap found in 0.2 sec, 45001 overlaps total in 03:25:50, total exceptions 0
    Processed row 46001/56589, 1 overlap found in 0.3 sec, 46001 overlaps total in 03:30:20, total exceptions 0
    Processed row 47001/56589, 1 overlap found in 0.2 sec, 47001 overlaps total in 03:34:45, total exceptions 0
    Processed row 48001/56589, 1 overlap found in 0.3 sec, 48001 overlaps total in 03:39:02, total exceptions 0
    Processed row 49001/56589, 1 overlap found in 0.3 sec, 49001 overlaps total in 03:43:24, total exceptions 0
    Processed row 50001/56589, 1 overlap found in 0.2 sec, 50001 overlaps total in 03:47:42, total exceptions 0
    Processed row 51001/56589, 1 overlap found in 0.3 sec, 51001 overlaps total in 03:52:01, total exceptions 0
    Processed row 52001/56589, 1 overlap found in 0.3 sec, 52001 overlaps total in 03:56:22, total exceptions 0
    Processed row 53001/56589, 1 overlap found in 0.3 sec, 53001 overlaps total in 04:00:44, total exceptions 0
    Processed row 54001/56589, 1 overlap found in 0.3 sec, 54001 overlaps total in 04:05:03, total exceptions 0
    Processed row 55001/56589, 1 overlap found in 0.2 sec, 55001 overlaps total in 04:21:02, total exceptions 0
    Processed row 56001/56589, 1 overlap found in 0.4 sec, 56001 overlaps total in 08:05:59, total exceptions 0
    Processed row 56589/56589, 1 overlap found in 0.3 sec, 56589 overlaps total in 17:16:55, total exceptions 0
    Overlap processing complete

In [ ]:
gdf = gdf_CA_FSA
cities = []
citygroup = 'Canada'

gdf.merge(gdf_union,on='DAUID')

for cityname in citynames_CA:
    if not cityname in [city['name'] for city in cities]:
        cities.append({'name':cityname, 'group':citygroup})

for city in getCityGroup(citygroup):
    city_da = []
    for c in cities_DA:
        if c['name']==city['name']:
            city_da = c
    updateCity(city, gdf_CA_FSA.loc[,:])
    fillCity(city,gdf,'CTUID','equals',city_da['gdf']['DAUID'].unique().tolist())

# Remove two large areas in Halifax
#updateCity(getCityByName('Halifax'),getCityByName('Halifax')['gdf'].loc[~((tmp_gdf['ADAUID']==12090050) | (tmp_gdf['ADAUID']==12090010)),:])

cities_FSA = cities

tooltip_FSA = ['PRNAME','CMANAME','CTUID']
mapCitiesAdjacent(getCityGroup(citygroup),'Area','Canadian Cities, Census Tracts colored by square meters',tooltip_FSA)

In [ ]:
gdf11 = gdf_CA_FSA.copy(deep=True)
gdf22 = gdf_CA_DA.copy(deep=True)

In [ ]:
ratio1 = gdf1['Geometry'].area/gdf11['Geometry'].area - 1
ratio1.unique()

In [ ]:
ratio2 = gdf2['Geometry'].area/gdf22['Geometry'].area - 1
ratio2.unique()

In [ ]:
ratio2df = gdf2[['Geometry']].area/gdf22[['Geometry']].area - 1
ratio2df.unique()

In [ ]:
ratio2short = ratio2.loc[abs(ratio2)>1e-6]
ratio2short

In [ ]:
print(len(gdf2.loc[50956,'Geometry']))
print(len(gdf2.loc[50956,'Geometry'].to_wkt()))
len1 = [len(g.to_wkt()) for g in gdf1['Geometry']]
len2 = [len(g.to_wkt()) for g in gdf2['Geometry']]
len11 = [len(g.to_wkt()) for g in gdf11['Geometry']]
len22 = [len(g.to_wkt()) for g in gdf22['Geometry']]

In [ ]:
import statistics
print(statistics.mean(len1))
print(statistics.mean(len2))
print(max(len1))
print(max(len2))
print(len2[50956])
print()
print(np.array(times)[np.where(np.array(times)>10)[0]])
#print(times[np.where(np.array(times)>10)[:,0]])
#print(pd.Series(times).loc[np.argwhere(pd.Series(times)>1)])
#plt.plot(times,len2,'.')
#plt.xlabel('Time [s]')
#plt.ylabel('Length [char]')
#plt.xlim([0,100])
#plt.ylim([1,10000000])
plt.plot(range(len(times)),np.log10([t for t, l in zip(times,len2)]),'.')
plt.xlabel('Index')
plt.ylabel('Log10(Time)')

We can look at the difference in geometry between original and .buffer(0) versions through the length of the geometry entry.

In [ ]:
len1 = [len(g.to_wkt()) for g in gdf1['Geometry']]
len2 = [len(g.to_wkt()) for g in gdf2['Geometry']]
len1b = [len(g.to_wkt()) for g in gdf1b['Geometry']]
len2b = [len(g.to_wkt()) for g in gdf2b['Geometry']]

In [ ]:
len_ratio1 = np.array(len1)/len1b
len_ratio2 = np.array(len2)/len2b
plt.plot(len_ratio1)
plt.plot(len_ratio2)
plt.ylabel('Original:Buffered WKT Length')
plt.xlabel('Element Index')
print(f'There are {sum(len_ratio1<1)} FSAs with increased geometries, {sum(len_ratio1>1)} with reduced geometries')
print(f'There are {sum(len_ratio2<1)} DAs with increased geometries, {sum(len_ratio2>1)} with reduced geometries')

In [ ]:
ratio2.unique()

New code here for FSA correspondence using digital boundary files:

In [ ]:
%%time
gdf_CA_FSA_D = GMLtoGDF('lfsa000a16g_e.gml')
gdf_CA_DA_D = GMLtoGDF('lda_000a16g_e.gml')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import logging
logging.getLogger('shapely.geos').setLevel(logging.CRITICAL)

In [ ]:
gdf1 = gdf_CA_FSA_D.copy(deep=True)
gdf1b = gdf1.copy(deep=True)
keyfield1 = 'CFSAUID'
gdf2 = gdf_CA_DA_D.copy(deep=True)
gdf2b = gdf2.copy(deep=True)
keyfield2 = 'DAUID'

gdf_union = geopandas.GeoDataFrame()
times = []
areas = []

from shapely.geometry import shape, mapping

start_time = time.time()
gdf1b['Geometry'] = gdf1b['Geometry'].buffer(0)
print(f'Polygon conversion for {keyfield1} completed in {time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))}')

start_time = time.time()
gdf2b['Geometry'] = gdf2b['Geometry'].buffer(0)
print(f'Polygon conversion for {keyfield2} completed in {time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))}')

exceptioncount = 0
start_time = time.time()
for i in range(gdf2.shape[0]):
    loop_start = time.time()
    try:
        gdf_tmp = gdf1['Geometry'].intersection(gdf2['Geometry'].iloc[i]) # This takes about 0.3 seconds per execution
    except (shapely.errors.TopologicalError): # This sometimes occurs
        #print(f'Handling exception at index {i}')
        exceptioncount += 1
        gdf_tmp = gdf1b['Geometry'].intersection(gdf2b['Geometry'].iloc[i]) # This may corrupt the area but usually works as fast as the original and reliably does not fail

    areas.append(gdf_tmp.area)
    ind = np.argmax(areas[-1]) # The FSA boundaries respect DA boundaries according to the , so there is just one FSA associated with each DA, which should have significantly larger area than any other.
    gdf_tmp = geopandas.GeoSeries(gdf_tmp.iloc[ind],crs=gdf1['Geometry'].crs)
    gdf_tmp = geopandas.GeoDataFrame(geometry=gdf_tmp,crs=gdf_tmp.crs)
    gdf_tmp[keyfield1] = gdf1[keyfield1].iloc[ind]
    gdf_tmp[keyfield2] = gdf2[keyfield2].iloc[i]
    gdf_union = gdf_union.append(gdf_tmp,ignore_index=True)
    loop_end = time.time()
    times.append(loop_end-loop_start)
    if i%1000==0 or i+1==gdf2.shape[0]:
        print(f'Processed row {i+1}/{gdf2.shape[0]}, {gdf_tmp.shape[0]} overlap found in {loop_end-loop_start:.1f} sec, {gdf_union.shape[0]} overlaps total in {time.strftime("%H:%M:%S", time.gmtime(loop_end-start_time))}, total exceptions {exceptioncount}')
print('Overlap processing complete')

Polygon conversion for CFSAUID completed in 00:00:01
Polygon conversion for DAUID completed in 00:00:04
Processed row 1/56590, 1 overlap found in 0.1 sec, 1 overlaps total in 00:00:00, total exceptions 0
Processed row 1001/56590, 1 overlap found in 0.1 sec, 1001 overlaps total in 00:02:19, total exceptions 13
Processed row 2001/56590, 1 overlap found in 0.2 sec, 2001 overlaps total in 00:04:54, total exceptions 17
Processed row 3001/56590, 1 overlap found in 0.1 sec, 3001 overlaps total in 00:07:22, total exceptions 17
Processed row 4001/56590, 1 overlap found in 0.1 sec, 4001 overlaps total in 00:09:39, total exceptions 17
Processed row 5001/56590, 1 overlap found in 0.1 sec, 5001 overlaps total in 00:11:37, total exceptions 35
Processed row 6001/56590, 1 overlap found in 0.1 sec, 6001 overlaps total in 00:13:30, total exceptions 111
Processed row 7001/56590, 1 overlap found in 0.1 sec, 7001 overlaps total in 00:15:22, total exceptions 171
Processed row 8001/56590, 1 overlap found in 0.1 sec, 8001 overlaps total in 00:17:20, total exceptions 171
Processed row 9001/56590, 1 overlap found in 0.1 sec, 9001 overlaps total in 00:19:15, total exceptions 207
Processed row 10001/56590, 1 overlap found in 0.1 sec, 10001 overlaps total in 00:21:07, total exceptions 295
Processed row 11001/56590, 1 overlap found in 0.1 sec, 11001 overlaps total in 00:23:00, total exceptions 472
Processed row 12001/56590, 1 overlap found in 0.1 sec, 12001 overlaps total in 00:24:52, total exceptions 472
Processed row 13001/56590, 1 overlap found in 0.1 sec, 13001 overlaps total in 00:26:34, total exceptions 472
Processed row 14001/56590, 1 overlap found in 0.1 sec, 14001 overlaps total in 00:28:22, total exceptions 472
Processed row 15001/56590, 1 overlap found in 0.1 sec, 15001 overlaps total in 00:30:08, total exceptions 472
Processed row 16001/56590, 1 overlap found in 0.1 sec, 16001 overlaps total in 00:31:59, total exceptions 473
Processed row 17001/56590, 1 overlap found in 0.1 sec, 17001 overlaps total in 00:33:53, total exceptions 474
Processed row 18001/56590, 1 overlap found in 0.1 sec, 18001 overlaps total in 00:35:58, total exceptions 475
Processed row 19001/56590, 1 overlap found in 0.1 sec, 19001 overlaps total in 00:37:56, total exceptions 476
Processed row 20001/56590, 1 overlap found in 0.1 sec, 20001 overlaps total in 00:39:46, total exceptions 476
Processed row 21001/56590, 1 overlap found in 0.1 sec, 21001 overlaps total in 00:41:31, total exceptions 476
Processed row 22001/56590, 1 overlap found in 0.2 sec, 22001 overlaps total in 00:43:20, total exceptions 477
Processed row 23001/56590, 1 overlap found in 0.1 sec, 23001 overlaps total in 00:45:09, total exceptions 477
Processed row 24001/56590, 1 overlap found in 0.1 sec, 24001 overlaps total in 00:46:58, total exceptions 477
Processed row 25001/56590, 1 overlap found in 0.1 sec, 25001 overlaps total in 00:48:45, total exceptions 477
Processed row 26001/56590, 1 overlap found in 0.1 sec, 26001 overlaps total in 00:50:33, total exceptions 477
Processed row 27001/56590, 1 overlap found in 0.1 sec, 27001 overlaps total in 00:52:22, total exceptions 478
Processed row 28001/56590, 1 overlap found in 0.1 sec, 28001 overlaps total in 00:54:09, total exceptions 479
Processed row 29001/56590, 1 overlap found in 0.1 sec, 29001 overlaps total in 00:55:56, total exceptions 479
Processed row 30001/56590, 1 overlap found in 0.1 sec, 30001 overlaps total in 00:57:42, total exceptions 479
Processed row 31001/56590, 1 overlap found in 0.1 sec, 31001 overlaps total in 00:59:31, total exceptions 479
Processed row 32001/56590, 1 overlap found in 0.1 sec, 32001 overlaps total in 01:01:25, total exceptions 479
Processed row 33001/56590, 1 overlap found in 0.1 sec, 33001 overlaps total in 01:03:15, total exceptions 480
Processed row 34001/56590, 1 overlap found in 0.1 sec, 34001 overlaps total in 01:05:06, total exceptions 480
Processed row 35001/56590, 1 overlap found in 0.1 sec, 35001 overlaps total in 01:07:06, total exceptions 481
Processed row 36001/56590, 1 overlap found in 0.1 sec, 36001 overlaps total in 01:09:03, total exceptions 484
Processed row 37001/56590, 1 overlap found in 0.1 sec, 37001 overlaps total in 01:11:06, total exceptions 517
Processed row 38001/56590, 1 overlap found in 0.1 sec, 38001 overlaps total in 01:13:13, total exceptions 607
Processed row 39001/56590, 1 overlap found in 0.1 sec, 39001 overlaps total in 01:15:33, total exceptions 747
Processed row 40001/56590, 1 overlap found in 0.2 sec, 40001 overlaps total in 01:18:16, total exceptions 876
Processed row 41001/56590, 1 overlap found in 0.1 sec, 41001 overlaps total in 01:21:24, total exceptions 1118
Processed row 42001/56590, 1 overlap found in 0.1 sec, 42001 overlaps total in 01:24:29, total exceptions 1190
Processed row 43001/56590, 1 overlap found in 0.1 sec, 43001 overlaps total in 01:27:36, total exceptions 1236
Processed row 44001/56590, 1 overlap found in 0.1 sec, 44001 overlaps total in 01:30:12, total exceptions 1320
Processed row 45001/56590, 1 overlap found in 0.1 sec, 45001 overlaps total in 01:32:26, total exceptions 1354
Processed row 46001/56590, 1 overlap found in 0.2 sec, 46001 overlaps total in 01:34:42, total exceptions 1355
Processed row 47001/56590, 1 overlap found in 0.1 sec, 47001 overlaps total in 01:37:05, total exceptions 1356
Processed row 48001/56590, 1 overlap found in 0.1 sec, 48001 overlaps total in 01:39:18, total exceptions 1359
Processed row 49001/56590, 1 overlap found in 0.2 sec, 49001 overlaps total in 01:41:24, total exceptions 1361
Processed row 50001/56590, 1 overlap found in 0.1 sec, 50001 overlaps total in 01:43:32, total exceptions 1364
Processed row 51001/56590, 1 overlap found in 0.1 sec, 51001 overlaps total in 01:45:23, total exceptions 1364
Processed row 52001/56590, 1 overlap found in 0.1 sec, 52001 overlaps total in 01:47:14, total exceptions 1364
Processed row 53001/56590, 1 overlap found in 0.1 sec, 53001 overlaps total in 01:49:13, total exceptions 1364
Processed row 54001/56590, 1 overlap found in 0.1 sec, 54001 overlaps total in 01:51:15, total exceptions 1365
Processed row 55001/56590, 1 overlap found in 0.1 sec, 55001 overlaps total in 01:53:22, total exceptions 1365
Processed row 56001/56590, 1 overlap found in 0.1 sec, 56001 overlaps total in 01:55:39, total exceptions 1366
Processed row 56590/56590, 1 overlap found in 0.3 sec, 56590 overlaps total in 01:57:35, total exceptions 1368
Overlap processing complete

import dill
with open('GDF_FSA-DA-D.db','wb') as file:
    dill.dump(gdf_union,file)
with open('GDF_FSA-DA-D_times.db','wb') as file:
    dill.dump(times,file)
with open('GDF_FSA-DA-D_areas.db','wb') as file:
    dill.dump(areas,file)

import dill
with open('GDF_FSA-DA-D.db','r') as file:
    gdf_union = dill.load(file)
with open('GDF_FSA-DA-D_times.db','r') as file:
    times = dill.load(file)
with open('GDF_FSA-DA-D_areas.db','r') as file:
    areas = dill.load(file)

Processing overlap with digital as opposed to geographic boundary files goes generally three times faster (probably due to many island and river boundaries in the latter), and all segments seem to take around 2 minutes, indicating no significant anomalous delays.  Let's see if there are any processing time outliers.

In [ ]:
plt.plot(range(len(times)),np.log10([t for t, l in zip(times,len2)]),'.')
plt.xlabel('Index')
plt.ylabel('Log$_{10}$(Time [s])');

We can look at the difference in geometry between original and .buffer(0) versions through the length of the geometry entry.

In [ ]:
len1 = [len(g.to_wkt()) for g in gdf1['Geometry']]
len2 = [len(g.to_wkt()) for g in gdf2['Geometry']]
len1b = [len(g.to_wkt()) for g in gdf1b['Geometry']]
len2b = [len(g.to_wkt()) for g in gdf2b['Geometry']]

In [ ]:
len_ratio1 = np.array(len1)/len1b
len_ratio2 = np.array(len2)/len2b
plt.plot(len_ratio1)
plt.plot(len_ratio2)
plt.ylabel('Original:Buffered WKT Length')
plt.xlabel('Element Index')
print(f'There are {len(len_ratio1)} FSAs with {sum(len_ratio1<1)} increased geometries and {sum(len_ratio1>1)} reduced geometries')
print(f'There are {len(len_ratio2)} DAs with {sum(len_ratio2<1)} increased geometries and {sum(len_ratio2>1)} reduced geometries')

This indicates minor differences; even with 1368 exceptions (out of 56590 DAs) during unbuffered geometry processing, these (likely) arise from only 9 FSAs and 53 DAs where the number of points in their geometry changes upon buffering.

We can also look at how the area of the geometries changes upon buffering.

In [ ]:
ratio1 = gdf1b['Geometry'].area/gdf1['Geometry'].area - 1
print(f'There are {sum(ratio1!=0)} indices where areas differ, with {len(ratio1.unique())-1} unique nonzero area difference fraction values.\nNonzero differences presented below, where area fraction = A(original)/A(buffered) - 1 :\nRatio : Area Frac. : Multiples\n------|------------|----------')
for i, v, m in zip(np.where(ratio1!=0)[0],ratio1[ratio1!=0],ratio1[ratio1!=0]/min(abs(ratio1[ratio1!=0]))):
    print(f'{i:>5} : {v: .3e} : {int(m): >3d}')

In [ ]:
ratio2 = gdf2b['Geometry'].area/gdf2['Geometry'].area - 1
print(f'There are {sum(ratio2!=0)} indices where areas differ, with {len(ratio2.unique())-1} unique nonzero area difference fraction values.\nNonzero differences presented below, where area fraction = A(original)/A(buffered) - 1 :\nRatio : Area Frac. : Multiples\n------|------------|----------')
for i, v, m in zip(np.where(ratio2!=0)[0],ratio2[ratio2!=0],ratio2[ratio2!=0]/min(abs(ratio2[ratio2!=0]))):
    print(f'{i:>5} : {v: .3e} : {int(m): >3d}')

These FSA area deviations are the same order of magnitude as the DA area deviations, within an order of magnitude of the area quantum.  There are 13 digits of precision in the GML file, but I have not yet investigated the precision of GeoDataFrames or Geopandas internal calculations (though note that buffer has a default resolution of 16, perhaps the area calculatio does also).



All of the above are very small area differences, compared to the ~1e-5 anomalies seen when using the geographic boundary files.

In [ ]:
a = pd.DataFrame(areas)
display(a.head(3))
print(a.shape)
print(gdf1.shape[0])
print(gdf2.shape[0])

In [ ]:
#display(a.head(20))
asorted = np.array(areas)
asorted.sort(axis=1)
#display(pd.DataFrame(asorted).head(20))
aratio = asorted[:,-2]/asorted[:,-1]
#aratio[0:20]

In [ ]:
exponent = np.logspace(-10,0,11)
counts = [len(np.where(aratio>x)[0]) for x in exponent]
print('Ratio : Count')
print('-------------')
for x, c in zip(exponent, counts):
    print(x,':',c)

In [ ]:
exponent = np.logspace(-10,0,11)
counts = [len(np.where(areas[:]>x)[0]) for x in exponent]
print('Area  : Count')
print('------|------')
for x, c in zip(exponent, counts):
    print(x,':',c)

In [ ]:
rind = np.where(aratio>1e-3)[0]
rval = aratio[rind]
print('Index : Ratio')
print('-------------')
for i, v in zip(rind,rval):
    print(f'{i: >5}',':',f'{v:.3f}')

In [ ]:
gdf_CA_DA_D.shape

Let's look at one of the suspect geometries:

In [ ]:
ind = np.where(areas[geom_index]>1e-10)[0]
print(ind)
print(areas[geom_index][ind])

So use ratio to determine cutoff for indices for DA
Plot da's with area as a thing

function to get DAs from a FSA
function to get FSAs from a DA

In [ ]:
geom_index = 53335

fsa = gdf_union.loc[geom_index,'CFSAUID']
das = areas[geom_index]

# create a plain world map
geom_centroid = gdf_union.iloc[geom_index,:].geometry.centroid
geom_centroid = [geom_centroid.y,geom_centroid.x]

geom_map = folium.Map(location=geom_centroid, control_scale=True)
#geom_map.fit_bounds(cities[city_index]['bounds'])

folium.Choropleth(
    geo_data=gdf_union.iloc[[geom_index]].to_json(),
    data=gdf_union,
    columns=['CFSAUID', 'DAUID'],
    key_on='feature.properties.CFSAUID',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
).add_to(geom_map)

folium.Choropleth(
    geo_data=gdf_CA_DA_D.iloc[[geom_index]].to_json(),
    data=gdf_union,
    columns=['CFSAUID', 'DAUID'],
    key_on='feature.properties.DAUID',
    fill_color='GnBu', 
    fill_opacity=0.7, 
    line_opacity=0.2,
).add_to(geom_map)

# display map
print(f'FSA: {fsa}')
display(geom_map)

Old code below.

In [ ]:
%%time
gdf_CA_FSA = GMLtoGDF('lfsa000b16g_e.gml')

#### Comparison of Areas

Let's compare some metrics for the various ways of dividing up the regions of interest.

First, let's look at the total number of regions for each dividing method:

In [ ]:
print('DA Count per city:')
print(count_DA)
print('ADA Count per city:')
print(count_ADA)
print('CT Count per city:')
count_CT = [(city['name'], city['gdf'].shape[0]) for city in cities]
print(count_ADA)
print('DA/CT Ratio per city:')
print([(a[0],float(f'{a[1]/b[1]:0.3}')) for a, b, in zip(count_DA,count_CT)])
print('ADA/CT Ratio per city:')
print([(a[0],float(f'{a[1]/b[1]:0.3}')) for a, b, in zip(count_ADA,count_CT)])

There are roughly twice as many Census Tracts as Aggregate Dissemination Areas in each city, with a corresponding smaller area.

We can also look at the areas occupied by each division.  The target area is:

In [ ]:
radius = 250
a = np.pi*radius**2
print(f'For radius {radius} meters, target area is {a:.0f} square meters (log10(area) = {np.log10(a):0.3})')
radius = 500
a = np.pi*radius**2
print(f'For radius {radius} meters, target area is {a:.0f} square meters (log10(area) = {np.log10(a):0.3})')

In [ ]:
def histCities(cities,featurename,regiontype=None):
    if type(cities)==dict:
        cities = [cities]
    if type(regiontype)==str:
        regiontype=[regiontype]*len(cities)
    n = len(cities)
    plt.figure(figsize=(4*n,3))
    bounds = extendBounds([0, (getCityBounds(cities,featurename))[1]])
    for i, city in enumerate(cities):
        plt.subplot(1,n,i+1)
        plt.hist(city['gdf'][featurename],bins=np.arange(0,bounds[1],extendBounds([0, bounds[1]/10])[1]))
        plt.xlabel(featurename)
        plt.ylabel(f'Number of {"Regions" if regiontype==None else regiontype[i]}')
        plt.title(f"{city['name']}, {city['gdf'].shape[0]} {'Regions' if regiontype==None else regiontype[i]}")
        plt.xticks(np.arange(0,bounds[1],extendBounds([0, bounds[1]/10])[1]))

def histCitiesLog10(cities,featurename,regiontype=None):
    if type(cities)==dict:
        cities = [cities]
    if type(regiontype)==str:
        regiontype=[regiontype]*len(cities)
    n = len(cities)
    plt.figure(figsize=(4*n,4))
    bounds = np.log10(extendBounds(getCityBounds(cities,featurename),'nearestPower',10))
    span = bounds[1]-bounds[0]
    step = 1
    labelstep = 1
    while span/step<10:
        if np.log10(step)%1==0: # leading digit is 1
            labelstep = step
            step=step/2
        elif np.log10(step)%1>0.5: # leading digit is 5
            labelstep = step
            step=step*2/5
        else: # leading digit is 2
            labelstep = step
            step=step/2
    width = 0.8*step
    for i, city in enumerate(cities):
        plt.subplot(1,n,i+1)
        plt.hist(city['gdf'][featurename].apply(np.log10),bins=np.arange(bounds[0]-width/2,bounds[1]+width/2,step),width=0.8*step)
        plt.xlabel(f'Log$_{{{10}}}$({featurename})')
        plt.ylabel(f'Number of {"Regions" if regiontype==None else regiontype[i]}')
        plt.title(f"{city['name']}, {city['gdf'].shape[0]} {'Regions' if regiontype==None else regiontype[i]}")
        plt.xticks(np.arange(bounds[0],bounds[1],labelstep))
        plt.gca().add_patch(plt.Rectangle((5.29,0), 0.61, plt.ylim()[1], color='red', alpha=0.3))
        
    plt.tight_layout()

We now plot the distribution of areas for each city when considering different area types.  The red area is bounded by areas corresponding to circular areas with diameter of 500 meters and radius of 500 meters.

In [ ]:
histCitiesLog10(cities_DA,'Area','Dissemination Areas')

In [ ]:
histCitiesLog10(cities_ADA,'Area','Agg. Dissemination Areas')

In [ ]:
histCitiesLog10(cities_CT,'Area','Census Tracts')

In [ ]:
histCitiesLog10(cities_FSA,'Area','Forward Sortation Areas')

From this we can see quantitatively that CTs best match our target area range, while ADAs are larger and DAs smaller.

## Census Data

File T1901EN.CSV was downloaded from the [Canadian Census Data](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm) site

<hr>

## Appendix

### Osgeo package

Before finding the geopandas library, I was using [gdal](https://gdal.org/python/index.html) (which is one of many dependencies of geopandas).  Using gdal's ogr and osr modules in the osgeo package I read in the gml file, converted coordinates to accurately compute area, and converted coordinates again to latitude and longitude.  As a guide to learning the library I adapted code from some [examples](https://pcjericks.github.io/py-gdalogr-cookbook/geometry.html#quarter-polygon-and-create-centroids).  Conversion to geojson may be done explicitly as demonstrated [here](https://gis.stackexchange.com/questions/77974/converting-gml-to-geojson-using-python-and-ogr-with-geometry-transformation), but by the time I was going to convert to geojson I found it much easier with geopandas, and this approach was abandoned.

#### Parsing GML file

    from osgeo import ogr
    from osgeo import osr
    
    def parseGMLtoDF(fn_gml,limit=None):
        '''Reads in a GML file and outputs a DataFrame

        Adds columns for Geometry (in WKT format), Latitude (of centroid), Longitude (of centroid) and Area (in square meters)
        '''
        # Add lightweight progress bar, source copied from GitHub
        import ipypb

        # Read in the file
        source = ogr.Open(fn_gml)
        layer = source.GetLayer(0) # there is only one Layer (the FeatureCollection)

        # Get a list of field names to extract (also could be gotten from schema e.g. lda_000b16g_e.xsd)
        #   Fields are in order (sequence) and none are required (minOccurs=0)
        layerDefinition = layer.GetLayerDefn()
        layerFields = [layerDefinition.GetFieldDefn(i).GetName() for i in range(layerDefinition.GetFieldCount())]

        # Initialize the output dataframe
        df = pd.DataFrame(columns=[*layerFields, 'Geometry', 'Latitude', 'Longitude', 'Area'])

        # Extract data from the features
        parse_limit = layer.GetFeatureCount()
        if not limit is None:
            parse_limit = limit

        for i in ipypb.track(range(parse_limit)):
            # Get the feature to be processed
            feature = layer.GetNextFeature()

            # Copy all fields into an empty dataframe
            df_tmp = pd.DataFrame(columns=df.columns)

            # Copy all fields individually, in case some are missing
            for i, fieldname in enumerate(layerFields):
                if feature.IsFieldSet(i):
                    df_tmp.loc[0,fieldname] = feature.GetFieldAsString(fieldname)

            # Get the latitude and longitude
            inref = feature.GetGeometryRef().GetSpatialReference() # EPSG 3347
            llref = osr.SpatialReference()
            llref.ImportFromEPSG(4326) # coordinates for this geometry are latitude, longitude
            lltransform = osr.CoordinateTransformation(inref, llref)
            geom = feature.GetGeometryRef().Clone()
            geom.Transform(lltransform)
            centroid = geom.Centroid()
            df_tmp['Latitude'] = centroid.GetX()
            df_tmp['Longitude'] = centroid.GetY()
            df_tmp['Geometry'] = geom.ExportToWkt()

            # Get the area by converting to a locally-appropriate coordinate system
            inref = feature.GetGeometryRef().GetSpatialReference()
            arref = osr.SpatialReference()
            arref.ImportFromEPSG(6931) # TODO: let this choose appropriate projection based on centroid for full-globe applicability, or alter to be a geodesic calculation.  Lambert Cylindrical Equal Area 6933, U.S. National Atlas Equal Area Projection 2163, NSIDC EASE-Grid North 3408, WGS 84 / NSIDC EASE-Grid 2.0 North 6931, WGS 84 / NSIDC EASE-Grid 2.0 South 6932, WGS 84 / NSIDC EASE-Grid 2.0 Global/Temperate 6933
            artransform = osr.CoordinateTransformation(inref, arref)
            geom = feature.GetGeometryRef().Clone() # Fresh instance to prevent error accumulation from multiple transforms
            geom.Transform(artransform)
            df_tmp['Area'] = geom.Area() # TODO: investigate proj module, related to osgeo, for geodesic area

            # Add the feature dataframe to the output
            df = df.append(df_tmp, ignore_index=True)

        return df

#### Viewing fields

    layerDefinition = s2.GetLayerDefn()

    for i in range(layerDefinition.GetFieldCount()):
        print(layerDefinition.GetFieldDefn(i).GetName(),f" \t",s2.GetFeature(0).GetFieldAsString(s2.GetFeature(0).GetFieldIndex(layerDefinition.GetFieldDefn(i).GetName())))

In [ ]:
df_CA_DA = parseGMLtoDF('lda_000b16g_e.gml')

In [ ]:
display(df_CA_DA.head(2))
print(df_CA_DA.shape)

#### Layered parsing code

From when I thought dataframe copying was the limiting step

    # Read in the file
    source = ogr.Open('lda_000b16g_e.gml')
    layer = source.GetLayer(0) # there is only one Layer (the FeatureCollection)

    # Get a list of field names to extract (see lda_000b16g_e.xsd)
    #   Fields are in order (sequence) and none are required (minOccurs=0)
    layerDefinition = layer.GetLayerDefn()
    layerFields = [layerDefinition.GetFieldDefn(i).GetName() for i in range(layerDefinition.GetFieldCount())]

    %%time
    import ipypb # Lightweight progress bar, source copied from GitHub

    # Initialize the output dataframe - REUSE FOR ALL OF CANADA
    df_CA_DA = pd.DataFrame(columns=[*layerFields, 'Geometry', 'Latitude', 'Longitude', 'Area'])

    # Only process a few entries
    parse_limit = 5

    # Setup for 2-step dataframe appending, should be ~O(n^(1+e)) instead of ~O(n^2)
    n_features = layer.GetFeatureCount()
    #max_n_merge = 1000
    n_merge = n_features**0.5//1
    #n_merge = min(n_features**0.5//1,max_n_merge) # How many rows to process before a merge
    df_tmp_collect = pd.DataFrame(columns=df_CA_DA.columns)

    # Extract data from the features
    layer.ResetReading()
    for i in ipypb.track(range(layer.GetFeatureCount())):
        # Get the feature to be processed
        feature = layer.GetNextFeature()

        # Copy all fields into an empty dataframe
        df_tmp = pd.DataFrame(columns=df_CA_DA.columns)

        # Copy all fields individually, in case some are missing
        for i, fieldname in enumerate(layerFields):
            if feature.IsFieldSet(i):
                df_tmp.loc[0,fieldname] = feature.GetFieldAsString(fieldname)

        # Get the latitude and longitude
        inref = feature.GetGeometryRef().GetSpatialReference() # EPSG 3347
        llref = osr.SpatialReference()
        llref.ImportFromEPSG(4326) # coordinates for this geometry are latitude, longitude
        lltransform = osr.CoordinateTransformation(inref, llref)
        geom = feature.GetGeometryRef().Clone()
        geom.Transform(lltransform)
        centroid = geom.Centroid()
        df_tmp['Latitude'] = centroid.GetX()
        df_tmp['Longitude'] = centroid.GetY()
        df_tmp['Geometry'] = geom.ExportToWkt()

        # Get the area by converting to a locally-appropriate coordinate system
        inref = feature.GetGeometryRef().GetSpatialReference()
        arref = osr.SpatialReference()
        arref.ImportFromEPSG(6931) # Lambert Cylindrical Equal Area 6933, U.S. National Atlas Equal Area Projection 2163, NSIDC EASE-Grid North 3408, WGS 84 / NSIDC EASE-Grid 2.0 North 6931, WGS 84 / NSIDC EASE-Grid 2.0 South 6932, WGS 84 / NSIDC EASE-Grid 2.0 Global/Temperate 6933
        artransform = osr.CoordinateTransformation(inref, arref)
        geom = feature.GetGeometryRef().Clone() # Fresh instance to prevent error accumulation from multiple transforms
        geom.Transform(artransform)
        df_tmp['Area'] = geom.Area()

        # Add the feature dataframe to the output
        #df_tmp_collect = df_tmp_collect.append(df_tmp, ignore_index=True)
        df_tmp_collect = pd.concat([df_tmp_collect, df_tmp], ignore_index=True)

        # Check if 2nd-step dataframe needs to be reset
        if df_tmp_collect.shape[0] >= n_merge:
            #df_CA_DA = df_CA_DA.append(df_tmp_collect, ignore_index=True)
            df_CA_DA = pd.concat([df_CA_DA,df_tmp_collect], ignore_index=True)
            df_tmp_collect = pd.DataFrame(columns=df_CA_DA.columns)

        if not parse_limit is None:
            parse_limit -= 1
            if parse_limit<=0:
                break
    #df_CA_DA = df_CA_DA.append(df_tmp_collect, ignore_index=True)
    df_CA_DA = pd.concat([df_CA_DA,df_tmp_collect], ignore_index=True)

##### Timing Notes

limit | n_merge | s/it | s total
---|---|---|---
500 | 1 | 0.13 | 1:06
500 | 2 | 0.13 | 1:02
500 | 10 | 0.12 | 1:01
500 | 50 | 0.12 | 0:59.2
500 | 237 | 0.11 | 0:57.5
500 | 237 | 0.13 | 1:03
1000 | 1 | 0.25 | 4:05
1000 | 237 | 0.23 | 3:52 # maybe this was max instead of min?
1000 | 1000 | 0.23 | 3:54
1000 | 1000 | 0.24 | 3:55  # pd.concat instead of dataframe.append
1000 | 237 | 0.22 | 3:38  # pd.concat instead of dataframe.append
1000 | - | 0.27 | 4:25  # original, no 2-levels
500  | - | 0.15 | 1:15  # original, no 2-levels
500 | 237 | 0.07 | 0:34.3  # pd.concat instead of dataframe.append
2000 | 237 | 0.07 | 2:13  # pd.concat instead of dataframe.append # Have not been resetting df_CA_DA!
4000 | 237 | 0.06 | 4:15  # pd.concat instead of dataframe.append # Resetting df from here on, and concat too, and GetNextFeature instead of GetFeature(int) which may have been the bottleneck then...
1000 | 237 | 0.06 | 1:01
500 | 237 | 0.06 | 0:31.3
1000 | 50 | 0.06 | 1:00
1000 | 1 | 0.06 | 1:04
1000 | 1 | 0.23 | 3:52  # Try once with GetFeature(i), yes, this was the culprit, it must parse anew each time.
4000 | 1 | 0.09 | 5:41
4000 | 5 | 0.06 | 4:11
4000 | 20 | 0.06 | 3:41
4000 | 50 | 0.06 | 3:54
4000 | 50 | 0.06 | 3:57
4000 | 100 | 0.06 | 3:58
4000 | 175 | 0.06 | 4:03
4000 | 237 | 0.06 | 4:08
4000 | 500 | 0.06 | 4:18
4000 | 500 | 0.06 | 3:50
4000 | 1000 | 0.06 | 4:16
4000 | 4000 | 0.07 | 4:58 # Oh, also hadn't reset the counter... adding that before last 500 and 30, that's been introducing variability too
4000 | 1 | 0.06 | 4:16
4000 | 30 | 0.06 | 4:17, 4:09 # Might change when tabs are changed (first with changing, second staying on same tab)
4000 | 60 | 0.06 | 3:53, 4:03
4000 | 200 | 0.06 | 3:56
4000 | 500 | 0.06 | 3:44
4000 | 1000 | 0.06 | 3:55
4000 | 4000 | 0.06 | 4:03
4000 | - | 0.06 | 4:04 # original, no 2-levels, with GetNextFeature

Gain is marginal with the double layering... might be important for very large sets, but at 4000 features results in at most a 10% speedup.  Note that geopandas speedup is ~4,600%.

#### Getting an appropriate EPSG projection

Using UTM projections (of which there are 30, each subtending 6 degrees of longitude) was a commonly proposed method for projection for area calculation.  I went with the simpler EPSG-6931/2/3, which is easier due to one zone encompassing all of Canada (though with unknown accuracy tradeoff).  This approach is included just for reference.

    srcSR = osr.SpatialReference()
    srcSR.ImportFromEPSG(4326) # WGS84 Geographic
    destSR   = osr.SpatialReference()

    lyr.ResetReading()
    for feat in lyr:
        geom = feat.GetGeometryRef()
        if not geom.IsEmpty():                 # make sure the geometry isn't empty
            geom.AssignSpatialReference(srcSR) # you only need to do this if the shapefile isn't set or is set wrong
            env = geom.GetEnvelope()           # get the Xmin, Ymin, Xmax, Ymax bounds
            CentX = ( env[0] + env[2] ) / 2    # calculate the centre X of the whole geometry
            Zone  = int((CentX + 180)/6) + 1   # see http://gis.stackexchange.com/questions/13291/computing-utm-zone-from-lat-long-point
            EPSG  = 32600 + Zone               # get the EPSG code from the zone and the constant 32600 (all WGS84 UTM North start with 326)
            destSR.ImportFromEPSG(EPSG)        # create the 'to' spatial reference
            geom.TransformTo(destSR)           # project the geometry
            print geom.GetArea()               # get the area in square metres

### Original df_loc (Canada) Description

[Statistics Canada](https://www.statcan.gc.ca/eng/start) provides several geographic divisions for reporting of census statistics:

Dissemination Area: DAs are small, relatively stable geographic unit composed of one or more adjacent dissemination blocks with an average population of 400 to 700 persons based on data from the previous Census of Population Program. It is the smallest standard geographic area for which all census data are disseminated.

Aggregate Dissemination Area: ADAs cover the entire country and, where possible, have a population count between 5,000 and 15,000 people, and respect provincial, territorial, census division (CD), census metropolitan area (CMA) and census agglomeration (CA) with census tract (CT) boundaries in effect for the 2016 Census.

Census Tract: CTs are small, relatively stable geographic areas that usually have a population of less than 10,000 persons, based on data from the previous Census of Population Program. They are located in census metropolitan areas and in census agglomerations that had a core population of 50,000 or more in the previous census.

Census Metropolitan Area/ Census Agglomeration: A CMA or CA is formed by one or more adjacent municipalities centred on a population centre (known as the core). A CMA must have a total population of at least 100,000 of which 50,000 or more must live in the core based on adjusted data from the previous Census of Population Program. A CA must have a core population of at least 10,000 also based on data from the previous Census of Population Program. To be included in the CMA or CA, other adjacent municipalities must have a high degree of integration with the core, as measured by commuting flows derived from data on place of work from the previous Census Program.

Census Subdivision: CSD is the general term for municipalities (as determined by provincial/territorial legislation) or areas treated as municipal equivalents for statistical purposes (e.g., Indian reserves, Indian settlements and unorganized territories).

Census Consolidated Subdivision:A CCS is a group of adjacent census subdivisions within the same census division. Generally, the smaller, more densely-populated census subdivisions (towns, villages, etc.) are combined with the surrounding, larger, more rural census subdivision, in order to create a geographic level between the census subdivision and the census division.

Census Division: CDs are a group of neighbouring municipalities joined together for the purposes of regional planning and managing common services (such as police or ambulance services).  Census divisions are intermediate geographic areas between the province/territory level and the municipality (census subdivision).

Economic Region: An ER is a grouping of complete census divisions (CDs), with one exception in Ontario, created as a standard geographic unit for analysis of regional economic activity.

Note: CSDNAME = 'Toronto' gives the same area as the FSAs, which makes sense as this is probably the definition of 'Toronto' 

The meaning of prefixes for NAME (N), UID (U), PUID (PU), TYPE (T), and CODE (C):
* DA U Dissemination Area unique identifier (composed of the 2-digit province/territory unique identifier followed by the 2-digit census division code and the 4-digit dissemination area code)
* PR U,N Province or territory
* CD U,N,T Census Division
* CCS U,N Census Consolidated Subdivision
* CSD U,N,T Census Subdivision
* ER U,N Economic Region
* SAC T,C Statistical Area Classification: Part of are a component of a census metropolitan area, a census agglomeration, a census metropolitan influenced zone or the territories?
* CMA U,PU,N,T Census Metropolitan Area or Census Agglomeration name, PU Uniquely identifies the provincial or territorial part of a census metropolitan area or census agglomeration (composed of the 2-digit province or territory unique identifier followed by the 3-digit census metropolitan area or census agglomeration unique identifier)
* CT U,N Census Tract within census metropolitan area/census agglomeration
* ADA U Aggregate dissemination area unique identifier

We will use census Distribution Areas as proxies for neighborhoods for cities in Canada.  In previous work where the Forward Sortation Areas (first three characters of the postal code) were used as neighborhood proxies, the sizes of many areas were quite large (several kilometers across) and therefore are likely internally non-homogeneous from a features perspective at the walking-distance (500 m) length scale.  To convert to neighborhood names we can look up the associated census tract as seen on [this](https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods) Wikipedia page.

File lda_000b16g_e.gml was downloaded from the [Statistics Canada: Boundary Files](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2016-eng.cfm) site.

Exploring the gml file and computing the area and centroid of the distribution areas can be done using the [geopandas module](https://geopandas.org/).  Geopandas builds upon [osgeo](https://gdal.org/python/index.html) which can also be used to explore and compute with the gml file, but in testing was 46 times faster due to vectorization of many calculations compared to a naive approach (see Appendix).

Latitude and Longitude need to be obtained from a projection with those units, e.g. [EPSG-4326](https://epsg.io/4326).  Area can be calculated from an equal-area projection, e.g. [EPSG-6931](https://epsg.io/6931) (though a geodesic area calculation would be more accurate for larger regions, that would have to come from an additional package such as [proj](https://proj.org/), but since all regions here are small such the curvature of the earth is negligible, and altitude indtroduces an additional error likely comparable or larger to the curvature error, we will proceed with a simpler equal-area projection calculation).  The geometry is saved as text in the [Well-Known Text (WKT)](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) representation.

#### Read in the Canadian data file: Original
    gdf = geopandas.read_file('lda_000b16g_e.gml')
    gdf.rename_geometry('Geometry', inplace=True) # Default geometry column name is 'geometry'; changed for consistent capitalization of columns
    gdf.set_geometry('Geometry') # Renaming is insufficient; this sets special variable gdf.geometry = gdf['Geometry']
    gdf['Area'] = gdf['Geometry'].to_crs(epsg=6931).area
    gdf['Centroid'] = gdf['Geometry'].centroid
    gdf['Geometry'] = gdf['Geometry'].to_crs(epsg=4326)
    gdf['Centroid'] = gdf['Centroid'].to_crs(epsg=4326) # Only the set geometry is converted with gdf.to_crs(); all other geometry-containing columns must be converted explicitly; here we convert all columns explicitly
    gdf['Centroid Latitude'] = gdf['Centroid'].geometry.y
    gdf['Centroid Longitude'] = gdf['Centroid'].geometry.x
    gdf.drop(columns = 'Centroid', inplace=True) # Because WKT Point cannot be serialized to JSON, we drop the Centroid column and keep only its float components
    gdf_CA_DA = gdf # Rename because we may be generating additional variables
    gdf_CA_DA.head(2)

In [ ]:
# Not used - this naive choropleth displays only one map and doesn't allow much scale customization

city_index = 1

# create a plain world map
city_map = folium.Map(location=cities[city_index]['centroid'], control_scale=True)
city_map.fit_bounds(cities[city_index]['bounds'])

folium.Choropleth(
    geo_data=cities[city_index]['geojson'],
    data=cities[city_index]['data'],
    columns=['DAUID', 'Area'],
    key_on='feature.properties.DAUID',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Area [square meters]'
).add_to(city_map)

# display map
city_map

In [ ]:
# Not used - links choropleth to a layercontrol, but does not get the legend on the correct layer

from branca.element import MacroElement

from jinja2 import Template

class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)  # noqa
        
# https://gitter.im/python-visualization/folium?at=5a36090a03838b2f2a04649d
print('Area Selection Criterion: CSDNAME contains the city name')
import branca.element as bre
from branca.colormap import LinearColormap

f = bre.Figure()
sf = [[],[],[]]
city_map = [[],[],[]]
colormap = [[],[],[]]
cp = [[],[],[]]
for i, city in enumerate(cities):
    sf[i] = f.add_subplot(1,len(cities),1+i)
    city_map[i] = folium.Map(location=city['centroid'], control_scale=True)
    sf[i].add_child(city_map[i])
    
    title_html = '''<h3 align="center" style="font-size:16px;charset=utf-8"><b>{}</b></h3>'''.format(city['name'])
    city_map[i].get_root().html.add_child(folium.Element(title_html))
    city_map[i].fit_bounds(city['bounds'])
    cp[i] = folium.Choropleth(geo_data=city['geojson'],
                                data=city['data'],
                                columns=['DAUID', 'Area'],
                                key_on='feature.properties.DAUID',
                                fill_color='YlOrRd',
                                fill_opacity=0.7, 
                                line_opacity=0.2
                               )
    city_map[i].add_child(cp[i])
    city_map[i].add_child(folium.map.LayerControl())
    city_map[i].add_child(BindColormap(cp[i],cp[i].color_scale))

    city['map'] = city_map[i]

display(f)

### Original FSA-DA overlap calculator

This code contains alternate gdf massaging options, along with some comments recording the effects of massaging on execution time.  This naive code ended up executing buffering on all DA geometries each time an intersection calculation failed, which was quite often, and led to very long computation times.  The code retained in the main section is a cleaned up version of this final code which computes buffering once at the beginning of execution, saving a lot of time.

In [ ]:
gdf1 = gdf_CA_FSA.copy(deep=True)
keyfield1 = 'CFSAUID'
gdf2 = gdf_CA_DA.copy(deep=True)
keyfield2 = 'DAUID'

gdf_union = geopandas.GeoDataFrame()
times = []
areas = []

start_time = time.time()

from shapely.geometry import shape, mapping
'''
def _set_precision(coords, precision):
    result = []
    try:
        return round(coords, int(precision))
    except TypeError:
        for coord in coords:
            result.append(_set_precision(coord, precision))
    return result

gdf1['Geometry'] = [shapely.ops.unary_union(g) for g in gdf1['Geometry']]
geo1 = [mapping(g) for g in gdf1['Geometry']]
for g in geo1:
    g['coordinates'] = _set_precision(g['coordinates'],8)
gdf1['Geometry'] = [shape(g) for g in geo1]'''

gdf1['Geometry'] = gdf1['Geometry'].buffer(0)
#gdf1['Geometry'] = [shapely.geometry.MultiPolygon(shapely.ops.polygonize(shapely.ops.unary_union(g))) for g in gdf1['Geometry']]
#gdf1['Geometry'] = [shapely.ops.unary_union(g) for g in gdf1['Geometry'].buffer(0)]
print(f'Polygon conversion for {keyfield1} completed in {time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))}')
start_time = time.time()
'''
gdf2['Geometry'] = [shapely.ops.unary_union(g) for g in gdf2['Geometry']]
geo2 = [mapping(g) for g in gdf2['Geometry']]
for g in geo2:
    g['coordinates'] = _set_precision(g['coordinates'],8)
gdf2['Geometry'] = [shape(g) for g in geo2]'''

gdf2['Geometry'] = gdf2['Geometry'].buffer(0)
#gdf2['Geometry'] = [shapely.geometry.MultiPolygon(shapely.ops.polygonize(shapely.ops.unary_union(g))) for g in gdf2['Geometry']]
#gdf2['Geometry'] = [shapely.ops.unary_union(g) for g in gdf2['Geometry'].buffer(0)]
print(f'Polygon conversion for {keyfield2} completed in {time.strftime("%H:%M:%S", time.gmtime(time.time()-start_time))}')

# Using unary_union only: [shapely.ops.unary_union(g) for g in gdf1['Geometry']]
# Processed row 1001/56589, 1 overlap found in 0.3 sec, 1001 overlaps total in 00:07:45, total exceptions 1
# Processed row 2001/56589, 1 overlap found in 0.3 sec, 2001 overlaps total in 00:16:04, total exceptions 3
# Processed row 3001/56589, 1 overlap found in 0.3 sec, 3001 overlaps total in 00:21:41, total exceptions 3
# Processed row 4001/56589, 1 overlap found in 0.3 sec, 4001 overlaps total in 00:27:00, total exceptions 3
# Processed row 5001/56589, 1 overlap found in 0.4 sec, 5001 overlaps total in 00:43:14, total exceptions 15
# Processed row 6001/56589, 1 overlap found in 75.4 sec, 6001 overlaps total in 01:25:24, total exceptions 52

# Using polygonize: [shapely.geometry.MultiPolygon(shapely.ops.polygonize(shapely.ops.unary_union(g))) for g in gdf1['Geometry']]
# Polygon conversion for CFSAUID completed in 00:02:07
# Polygon conversion for DAUID completed in 00:01:56
# Processed row 1001/56589, 1 overlap found in 0.3 sec, 1001 overlaps total in 00:28:54, total exceptions 22
# Processed row 2001/56589, 1 overlap found in 0.3 sec, 2001 overlaps total in 00:44:38, total exceptions 32

# Using _set_precision: https://gis.stackexchange.com/questions/188622/rounding-all-coordinates-in-shapely/276512
# Polygon conversion for CFSAUID completed in 00:00:28
# Polygon conversion for DAUID completed in 00:01:21
# Exception at 308, 414, 422,...
# Adding unary_union afterward:
# Polygon conversion for CFSAUID completed in 00:01:26
# Polygon conversion for DAUID completed in 00:01:49
# Exceptions at 414, 532, 890, 
# Processed row 1001/56589, 1 overlap found in 0.4 sec, 1001 overlaps total in 00:07:35, total exceptions 3
# Processed row 2001/56589, 1 overlap found in 0.2 sec, 2001 overlaps total in 00:14:49, total exceptions 6
# Switching order so unary union is first:
# Exceptions at 308, 414, 532, 567, 648...
# Processed row 1001/56589, 1 overlap found in 0.3 sec, 1001 overlaps total in 00:22:10, total exceptions 23

# Using buffered geometry only: gdf1['Geometry'] = gdf1['Geometry'].buffer(0)
# Processed row 1001/56589, 1 overlap found in 0.2 sec, 1001 overlaps total in 00:05:11, total exceptions 0
# Processed row 2001/56589, 1 overlap found in 0.2 sec, 2001 overlaps total in 00:10:10, total exceptions 0


# gdf11 = gdf_CA_FSA.copy(deep=True)
# gdf22 = gdf_CA_DA.copy(deep=True)
# ratio1 = gdf11['Geometry'].area/gdf1['Geometry'].area
# ratio1.unique()
#  array([1.])

exceptioncount = 0
start_time = time.time()
for i in range(gdf2.shape[0]):
    loop_start = time.time()
    # gdf_tmp = geopandas.overlay(gdf1[['Geometry']], gdf2[['Geometry']].iloc[[i],:], how='intersection') # This takes about 50 seconds per execution
    try:
        gdf_tmp = gdf1['Geometry'].intersection(gdf2['Geometry'].iloc[i]) # This takes about 0.3 seconds per execution
    except (shapely.errors.TopologicalError): # This sometimes occurs e.g. index 421/422 calling a self-intersection; not sure what to make of it
        print(f'Handling exception at index {i}')
        exceptioncount += 1
        gdf_tmp = gdf1['Geometry'].buffer(0).intersection(gdf2['Geometry'].iloc[i].buffer(0)) # This takes a long time again, e.g. 52 sec for row 423, but better than failure!
        # Actually, these errors are discussed online, e.g. https://github.com/Toblerity/Shapely/issues/599 and https://stackoverflow.com/questions/35110632/splitting-self-intersecting-polygon-only-returned-one-polygon-in-shapely
        # The errors arise when geometry lines cross when not at a node
        # The buffer(0) trick can actually cause dropping of polygons, hence we use it only as a fallback
        # For example, there were ~29 exceptions in the first 5000/56589 rows of gdf1, roughly doubling the expected computation time from 5 to 10 hours
        # As a solution we perform the unary union on both input datasets before the looped intersection; this takes only ~2 minutes and prevents those errors which double the time
    areas.append(gdf_tmp.area)
    ind = np.argmax(areas[-1]) # The FSA boundaries respect DA boundaries according to the , so there is just one FSA associated with each DA, which should have significantly larger area than any other.
    gdf_tmp = geopandas.GeoSeries(gdf_tmp.iloc[ind],crs=gdf1['Geometry'].crs)
    gdf_tmp = geopandas.GeoDataFrame(geometry=gdf_tmp,crs=gdf_tmp.crs)
    gdf_tmp[keyfield1] = gdf1[keyfield1].iloc[ind]
    gdf_tmp[keyfield2] = gdf2[keyfield2].iloc[i]
    gdf_union = gdf_union.append(gdf_tmp,ignore_index=True)
    loop_end = time.time()
    times.append(loop_end-loop_start)
    if i%1000==0 or i+1==gdf2.shape[0]:
        print(f'Processed row {i+1}/{gdf2.shape[0]}, {gdf_tmp.shape[0]} overlap found in {loop_end-loop_start:.1f} sec, {gdf_union.shape[0]} overlaps total in {time.strftime("%H:%M:%S", time.gmtime(loop_end-start_time))}, total exceptions {exceptioncount}')
print('Overlap processing complete')

### Original extendBounds Function and Testing

In [ ]:
def extendBounds(bounds,method='nearestLeadingDigit',scale=10):
    '''Extend bounds (low, high) to give round numbers for scalebars
    
    The low bound is decreased and the high bound is increased according to
        method to the first number satisfying the method conditions.
    Returns a new bound which includes the old bounds in its entirety
    
    Parameters
    ----------
    bounds: (low_bound, high_bound) list or tuple
    method: str describing the extension method
        'nearestLeadingDigit': Bounds are nearest numbers with leading digit followed by zeros
        'nearestPower': Bounds are nearest powers of scale (scale must be > 1).  For negative numbers, the sign and direction are reversed, the extension performed, then the sign of the result is reversed back.
        'nearestMultiple': Bounds are nearest multiple of scale (scale must be > 0)
        'round': Bounds are rounded
    scale: numeric as described in method options
    
    Returns
    -------
    2-element tuple of extended bounds e.g. (newlow,newhigh)
    '''
    if bounds[0]>bounds[1]:
        print('bounds must be ordered from least to greatest')
        return None    
    if method=='nearestLeadingDigit':
        iszero = np.array(bounds)==0
        isnegative = np.array(bounds) < 0
        offsets = [1 if isnegative[0] else 0, 0 if isnegative[1] else 1]
        power = [0 if z else np.floor(np.log10(abs(b))) for b, z in zip(bounds, iszero)]
        firstdigit = [abs(b)//np.power(10,p) for b, p in zip(bounds, power)]
        exceeds = [abs(b)>f*np.power(10,p) for b, f, p in zip(bounds, firstdigit, power)]
        newbounds = [abs(b) if not t else (f+o)*np.power(10,p) for b, t, n, f, o, p in zip(bounds, exceeds, isnegative, firstdigit, offsets, power)]
        newbounds = [-n if t else n for n, t in zip(newbounds,isnegative)]
    elif method=='nearestPower':
        try:
            scale = float(scale)
            if scale<=1:
                print('scale should be greater than 1')
                return None
        except ValueError:
            print('scale should be a number greater than 1')
            return None
        isnegative = np.array(bounds) < 0
        roundfuns = [np.ceil if isnegative[0] else np.floor, np.floor if isnegative[1] else np.ceil]
        newbounds = [0 if b==0 else np.power(scale, r(np.log10(abs(b))/np.log10(scale))) for b, r in zip(bounds,roundfuns)]
        newbounds = [-n if t else n for n, t in zip(newbounds,isnegative)]
    elif method=='nearestMultiple':
        try:
            scale = float(scale)
            if scale<=0:
                print('scale should be greater than 0')
                return None
        except ValueError:
            print('scale should be a number greater than 0')
            return None
        newbounds = [scale*(np.floor(bounds[0]/scale)), scale*(np.ceil(bounds[1]/scale))]
    elif method=='round':
        newbounds = [np.floor(bounds[0]), np.ceil(bounds[1])]
    else:
        print('Invalid method, see help(extendBounds)')
        return None
    return newbounds

In [ ]:
print("Testing invalid method")
print("  Expect errors:")
print(f"{extendBounds([11,130],'invalid')}")
print()

print("Testing method 'nearestLeadingDigit'")
print("  Expect errors:")
print(f"{extendBounds([9,-930],'nearestLeadingDigit')}")
print(f"{extendBounds([-9,-930],'nearestLeadingDigit')}")
print("  Expect success:")
print(f"{extendBounds([11,130],'nearestLeadingDigit')}",)
print(f"{extendBounds([11,130],'nearestLeadingDigit',-1)}")
print(f"{extendBounds([9,930],'nearestLeadingDigit')}")
print(f"{extendBounds([-9,930],'nearestLeadingDigit')}")
print(f"{extendBounds([-990,-930],'nearestLeadingDigit')}")
print(f"{extendBounds([-990,0.05],'nearestLeadingDigit')}")
print(f"{extendBounds([0,0.052],'nearestLeadingDigit')}")
print()

print("Testing method 'nearestPower'")
print("  Expect errors:")
print(f"{extendBounds([11,130],'nearestPower',-2)}")
print(f"{extendBounds([11,130],'nearestPower',0)}")
print(f"{extendBounds([11,130],'nearestPower',1)}")
print(f"{extendBounds([-11,-130],'nearestPower',10)}")
print("  Expect success:")
print(f"{extendBounds([11,130],'nearestPower')}")
print(f"{extendBounds([10,100],'nearestPower')}")
print(f"{extendBounds([11,130],'nearestPower',1.1)}")
print(f"{extendBounds([11,130],'nearestPower',2)}")
print(f"{extendBounds([11,130],'nearestPower',10)}")
print(f"{extendBounds([11,130],'nearestPower',10.)}")
print(f"{extendBounds([-11,130],'nearestPower',10)}")
print(f"{extendBounds([-5100,-130],'nearestPower',10)}")
print(f"{extendBounds([-.0101,-0.00042],'nearestPower',10)}")
print(f"{extendBounds([0,0.00042],'nearestPower',10)}")
print()

print("Testing method 'nearestMultiple'")
print("  Expect errors:")
print(f"{extendBounds([11,132],'nearestMultiple',-2)}")
print(f"{extendBounds([11,132],'nearestMultiple',0)}")
print(f"{extendBounds([0,-10],'nearestMultiple',100)}")
print("  Expect success:")
print(f"{extendBounds([11,132],'nearestMultiple')}")
print(f"{extendBounds([10,130],'nearestMultiple')}")
print(f"{extendBounds([11.55,132.55],'nearestMultiple',0.1)}")
print(f"{extendBounds([11.55,132.55],'nearestMultiple',1)}")
print(f"{extendBounds([11.55,132.55],'nearestMultiple',100)}")
print(f"{extendBounds([-11,132],'nearestMultiple',10)}")
print(f"{extendBounds([-1121,-132],'nearestMultiple',10)}")
print(f"{extendBounds([-10,-10],'nearestMultiple',10)}")
print(f"{extendBounds([-10,-10],'nearestMultiple',100)}")
print()

print("Testing method 'round'")
print("  Expect errors:")
print(f"{extendBounds([-11.1,-132.1],'round')}")
print("  Expect success:")
print(f"{extendBounds([11.1,132.1],'round')}")
print(f"{extendBounds([10,130],'round')}")
print(f"{extendBounds([11.1,132.1],'round',-2)}")
print(f"{extendBounds([-11.1,132.1],'round')}")
print(f"{extendBounds([-1100.1,-132.1],'round')}")

    Testing invalid method
      Expect errors:
    Invalid method, see help(extendBounds)
    None

    Testing method 'nearestLeadingDigit'
      Expect errors:
    bounds must be ordered from least to greatest
    None
    bounds must be ordered from least to greatest
    None
      Expect success:
    [10.0, 200.0]
    [10.0, 200.0]
    [9, 1000.0]
    [-9, 1000.0]
    [-1000.0, -900.0]
    [-1000.0, 0.05]
    [0, 0.06]

    Testing method 'nearestPower'
      Expect errors:
    scale should be greater than 1
    None
    scale should be greater than 1
    None
    scale should be greater than 1
    None
    bounds must be ordered from least to greatest
    None
      Expect success:
    [10.0, 1000.0]
    [10.0, 100.0]
    [10.834705943388395, 142.04293198443193]
    [8.0, 256.0]
    [10.0, 1000.0]
    [10.0, 1000.0]
    [-100.0, 1000.0]
    [-10000.0, -100.0]
    [-0.1, -0.0001]
    [0, 0.001]

    Testing method 'nearestMultiple'
      Expect errors:
    scale should be greater than 0
    None
    scale should be greater than 0
    None
    bounds must be ordered from least to greatest
    None
      Expect success:
    [10.0, 140.0]
    [10.0, 130.0]
    [11.5, 132.6]
    [11.0, 133.0]
    [0.0, 200.0]
    [-20.0, 140.0]
    [-1130.0, -130.0]
    [-10.0, -10.0]
    [-100.0, -0.0]

    Testing method 'round'
      Expect errors:
    bounds must be ordered from least to greatest
    None
      Expect success:
    [11.0, 133.0]
    [10.0, 130.0]
    [11.0, 133.0]
    [-12.0, 133.0]
    [-1101.0, -132.0]

In [ ]:
def getCityBounds(cities, featurename):
    '''Returns the global bounds of column featurename across all cities
    
    Parameters
    ----------
    cities: dict or list of dicts as defined above
    featurename: str column name in cities[i]['gdf'] for which bounds will be obtained
    
    Returns
    -------
    (min, max) value across all cities
    '''
    if type(cities)==dict:
        cities = [cities]
    bounds = [[],[]]
    for city in cities:
        bounds[0].append(city['gdf'][featurename].min())
        bounds[1].append(city['gdf'][featurename].max())
    bounds[0] = min(bounds[0])
    bounds[1] = max(bounds[1])
    return bounds

def extendBound(bound,direction='up',method='nearestLeadingDigit',scale=10):
    '''Extend bound to next 'round' number
    
    Parameters
    ----------
    bound: float or float castable number or a list thereof
    direction: {'up','down',nonzero number} or a list of these values indicating the direction to round in
    method: str describing the extension method
        'nearestLeadingDigit': Bound is nearest numbers with leading digit followed by zeros
        'nearestPower': Bound is nearest integer power of scale (scale must be > 1).  For negative numbers, the sign and direction are reversed, the extension performed, then the sign of the result is reversed back.
        'nearestMultiple': Bound is nearest multiple of scale (scale must be > 0)
        'round': Bound is rounded using the default method
    scale: numeric as described in method options or a list thereof
    
    Returns
    -------
    float: the extended bound
    
    Notes
    -----
    All inputs, if not single-valued, must be lists of length equal to input bound
    TODO: extend so that method may also be a list
    TODO: replace prints with raising errors
    '''
    import numpy as np
    
    # Check and adjust the length of inputs
    unlist_bound = False
    if not(type(bound) in {list,tuple,range}):
        bound = [bound]
        unlist_bound = True
    acceptable_len = set((1,len(bound)))
    if not(type(direction) in {list,tuple,range}):
        direction = [direction]
    if not(len(direction) in acceptable_len):
        print('"direction" must have length 1 or length equal to the length of "bound"')
        return None
    if (type(method) in {str}):
        method = [method]
    if not(len(method) in acceptable_len):
        print('"method" must have length 1 or length equal to the length of "bound"')
        return None
    if not(type(scale) in {list,tuple,range}):
        scale = [scale]
    if not(len(scale) in acceptable_len):
        print('"scale" must have length 1 or length equal to the length of "bound"')
        return None
    if len(bound)>1:
        if len(direction)==1: direction = [direction[0] for b in bound]
        if len(scale)==1: scale = [scale[0] for b in bound]
        
    # If multiple methods are specified, recursively call this function for each method and reassemble results
    if len(bound)>1 and len(method)>1:
        ret = np.array([None for b in bound])
        for m in list(set(method)):
            ind = np.where(np.array(method)==m)
            ret[ind] = extendBound(list(np.array(bound)[ind]),list(np.array(direction)[ind]),m,list(np.array(scale)[ind]))
        return list(ret)
    
    # Convert direction to a logical array roundup
    try:
        roundup = [True if d=='up' else False if d=='down' else True if float(d)>0 else False if float(d)<0 else None for d in direction]
    except:
        print('direction must be "up", "down", or a non-negative number')
        return None
    if any([r==None for r in roundup]):
        print('direction must be "up", "down", or a non-negative number')
        return None
    
    # Cases for multiple methods handled above, return to string method
    method = method[0]
    
    # Execute the conversions
    if method=='nearestLeadingDigit':
        iszero = np.array(bound)==0
        isnegative = np.array(bound) < 0
        offsets = np.logical_xor(roundup, isnegative)
        power = [0 if z else np.floor(np.log10(abs(b))) for b, z in zip(bound, iszero)]
        firstdigit = [abs(b)//np.power(10,p) for b, p in zip(bound, power)]
        exceeds = [abs(b)>f*np.power(10,p) for b, f, p in zip(bound, firstdigit, power)]
        newbound = [abs(b) if not t else (f+o)*np.power(10,p) for b, t, n, f, o, p in zip(bound, exceeds, isnegative, firstdigit, offsets, power)]
        newbound = [-n if t else n for n, t in zip(newbound,isnegative)]
    elif method=='nearestPower':
        try:
            scale = [float(s) for s in scale]
            if any([s<=1 for s in scale]):
                print('scale should be greater than 1')
                return None
        except ValueError:
            print('scale should be a number or list of numbers greater than 1')
            return None
        isnegative = np.array(bound) < 0
        offsets = np.logical_xor(roundup, isnegative)
        roundfuns = [np.ceil if o else np.floor for o in offsets]
        newbound = [0 if b==0 else np.power(s, r(np.log10(abs(b))/np.log10(s))) for b, r, s in zip(bound,roundfuns,scale)]
        newbound = [-n if t else n for n, t in zip(newbound,isnegative)]
    elif method=='nearestMultiple':
        try:
            scale = [float(s) for s in scale]
            if any([s<=0 for s in scale]):
                print('scale should be greater than 0')
                return None
        except ValueError:
            print('scale should be a number or list of numbers greater than 0')
            return None
        roundfuns = [np.ceil if r else np.floor for r in roundup]
        newbound = [s*(r(b/s)) for b, r, s in zip(bound,roundfuns,scale)]
    elif method=='round':
        roundfuns = [np.ceil if r else np.floor for r in roundup]
        newbound = [f(b) for b, f in zip(bound, roundfuns)]
    else:
        print('Invalid method, see help(extendBound)')
        return None
    return newbound[0] if unlist_bound else newbound

def extendBounds(bounds,method='nearestLeadingDigit',scale=10):
    if bounds[0]>bounds[1]:
        print('bounds must be ordered from least to greatest')
        return None    
    return extendBound(bounds,direction=['down','up'],method=method,scale=scale)